In [2]:
%run common.ipy

Using DW in /home/davo/Documents/GRASIA/daostack-notebooks/datawarehouse from date 2023-05-01T00:00:00
DW version 1.1.9
dao-analyzer version: 1.2.6
cache-scripts version: 1.1.9


In [3]:
import pandas as pd
import numpy as np
import numba

from tqdm.autonotebook import tqdm
tqdm.pandas()

from dao_analyzer.web.apps.daostack.data_access.daos.metric import srcs as DAOSTACK

# Local modules
%load_ext autoreload
%autoreload 1
%aimport utils

# Some global configs
pd.set_option('display.max_columns', 30)

/tmp/ipykernel_5591/1148700108.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Preparing DAOs df

In [4]:
# Organization info
dfd = pd.read_feather(DAOSTACK.DAOS)
dfh = pd.read_feather(DAOSTACK.REP_HOLDERS)
dfm = pd.read_feather(DAOSTACK.CACHE_PATH / 'reputationMints.arr')

print("Including DAOs from networks:", dfd.reset_index()['network'].unique())

dfd = dfd.astype({
    'id': 'string',
    'dao': 'string',
    'network': 'string',
}).infer_objects()
dfd = dfd.set_index(['network', 'dao'], verify_integrity=True)

# Activity
dfp = pd.read_feather(DAOSTACK.PROPOSALS, columns=['network', 'id', 'proposer', 'dao', 'createdAt', 'winningOutcome', 'boostedAt', 'stakesFor'])
dfv = pd.read_feather(DAOSTACK.VOTES, columns=['id', 'network', 'createdAt', 'voter', 'dao', 'proposal'])
dfs = pd.read_feather(DAOSTACK.STAKES, columns=['network', 'id', 'createdAt', 'dao', 'proposal'])
dfp['kind'] = 'proposal'
dfv['kind'] = 'vote'
dfs['kind'] = 'stake'

dfh['createdAt'] = pd.to_datetime(dfh['createdAt'], unit='s')
dfm['createdAt'] = pd.to_datetime(dfm['createdAt'], unit='s')

dfdm = dfd.reset_index().merge(dfm, left_on=['network', 'dao'], right_on=['network', 'dao'])

# Actual number of users
dfd['nusers'] = dfh.groupby(['network', 'dao'])['address'].nunique()
dfd['nusers'] = dfd['nusers'].fillna(0).astype(int)
# Historical number of users
dfd['hnusers'] = dfdm.groupby(['network', 'dao'])['address'].nunique()
dfd['hnusers'] = dfd['hnusers'].fillna(0)
# Get first user joined date
dfd['firstUser'] = dfdm.groupby(['network', 'dao'])['createdAt'].min()

DAO_ACTIVITY_COLS = ['network', 'dao', 'createdAt', 'kind']
dfact = pd.concat([x[DAO_ACTIVITY_COLS] for x in [dfp,dfv,dfs]])
dfact['createdAt'] = pd.to_datetime(dfact['createdAt'], unit='s')
dfact['createdAtMonth'] = dfact['createdAt'].dt.to_period('M')

# First and last activity date of each DAO
dfd['firstActivity'] = dfact.groupby(['network', 'dao'])['createdAt'].min()
dfd['lastActivity'] = dfact.groupby(['network', 'dao'])['createdAt'].max()
dfd['lastVote'] = dfact[dfact['kind'] == 'vote'].groupby(['network', 'dao'])['createdAt'].max()

# Activities count of each DAO
dfd['cntActivity'] = dfact.groupby(['network', 'dao'])['createdAt'].count()
dfd['nvotes'] = dfv.groupby(['network', 'dao'])['createdAt'].count()
dfd['nstakes'] = dfs.groupby(['network', 'dao'])['createdAt'].count()
dfd['nproposals'] = dfp.groupby(['network', 'dao'])['createdAt'].count()
dfd[['cntActivity', 'nvotes', 'nstakes', 'nproposals']] = dfd[['cntActivity', 'nvotes', 'nstakes', 'nproposals']].fillna(0).astype(int)

dfd['monthsSinceFirstUser'] = utils.getMonthsSince(dfd['firstUser'])
dfd['monthSinceFirstActivity'] = utils.getMonthsSince(dfd['firstActivity'])
dfd['monthsSinceLastActivity'] = utils.getMonthsSince(dfd['lastActivity'])
# Count months with some activity
dfd['monthsWithActivity'] = dfact.groupby(['network', 'dao'])['createdAtMonth'].nunique()
dfd['monthsWithProposals'] = dfact[dfact['kind'] == 'proposal'].groupby(['network', 'dao'])['createdAtMonth'].nunique()
dfd['pctActive'] = dfd['monthsWithActivity'] / dfd['monthSinceFirstActivity']

dfp['approved'] = dfp['winningOutcome'] == 'Pass'
dfp['staked'] = dfp['stakesFor'].astype(float) > 0
dfp['boosted'] = ~dfp['boostedAt'].isna()
dfd['pctApproved'] = dfp.groupby(['network', 'dao'])['approved'].mean()
dfd['pctBoosted'] = dfp.groupby(['network', 'dao'])['boosted'].mean()
dfd['pctStaked'] = dfp.groupby(['network', 'dao'])['staked'].mean()
dfd['vpp'] = dfv.groupby(['network', 'dao'])['id'].count() / dfp.groupby(['network', 'dao'])['id'].count()
dfd['spp'] = dfs.groupby(['network', 'dao'])['id'].count() / dfp.groupby(['network', 'dao'])['id'].count()

assert (dfd['pctActive'].isna() | dfd['pctActive'] <= 1).all(), 'A percentage can\'t be greater than 1'

# Classification report
dfp['TP'] =  dfp['boosted'] &  dfp['approved']
dfp['TN'] = ~dfp['boosted'] & ~dfp['approved']
dfp['FP'] =  dfp['boosted'] & ~dfp['approved']
dfp['FN'] = ~dfp['boosted'] &  dfp['approved']

dfd[['TP', 'TN', 'FP', 'FN']] = dfp.groupby(['network', 'dao'])[['TP', 'TN', 'FP', 'FN']].sum()
dfd['precision'] = (dfd['TP'] + dfd['TN']) / dfd[['TP', 'TN', 'FP', 'FN']].sum(axis=1)
dfd['precision_boosted'] = dfd['TP'] / (dfd['TP'] + dfd['FP'])
dfd['precision_nonboosted'] = dfd['TN'] / (dfd['TN'] + dfd['FN'])

dfd['nvoters'] = dfv.groupby(['network', 'dao'])['voter'].apply(utils.get_nunique_in_dao)
dfd['nproposers'] = dfp.groupby(['network', 'dao'])['proposer'].apply(utils.get_nunique_in_dao)

# An ethereum address has 20 bytes and we want just 8. 
dfd['shortid'] = dfd['id'].apply(lambda x: int(x[:10], base=16))
dfd['shortid'] = dfd['shortid'].astype('uint')
assert len(dfd) == dfd['shortid'].nunique(), "You can't just use 8 bytes to identify the DAOs"

# don't use category dtype, it gives some problems
# such as having to delete old category values
# and the dtype removing itself after picking
dfd['group'] = dfd['name'].replace('xDXdao', 'dxDAO')

# %pickledf dfact
%pickledf dfd

Including DAOs from networks: ['mainnet' 'xdai']
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 779 entries, ('mainnet', '0x006087d6ac20840c23ba298512db454a05c19b10') to ('xdai', '0xfff2b9bb869c7886a07c3105c7ed75fa48c4ff75')
Data columns (total 37 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   id                       779 non-null    string        
 1   name                     779 non-null    object        
 2   register                 779 non-null    object        
 3   nativeToken              779 non-null    object        
 4   nativeReputation         779 non-null    object        
 5   nusers                   779 non-null    int64         
 6   hnusers                  779 non-null    float64       
 7   firstUser                770 non-null    datetime64[ns]
 8   firstActivity            245 non-null    datetime64[ns]
 9   lastActivity             245 non-null    datetime64[ns]
 10  la

id  \
network dao                                                                                      
mainnet 0x006087d6ac20840c23ba298512db454a05c19b10  0x006087d6ac20840c23ba298512db454a05c19b10   
        0x00e1b6de09e01d5b178ecf68966a34bd1dcd4064  0x00e1b6de09e01d5b178ecf68966a34bd1dcd4064   
        0x00ea828bf150ec08cad91ee93841785c827b78f4  0x00ea828bf150ec08cad91ee93841785c827b78f4   
        0x01b46d83754ea98f3edd5e5ceb0aeb00cf291e8e  0x01b46d83754ea98f3edd5e5ceb0aeb00cf291e8e   
        0x02200bfe176e8e5f8dc58103afc8ca42ad863736  0x02200bfe176e8e5f8dc58103afc8ca42ad863736   
...                                                                                        ...   
xdai    0xfe525539277edd6769133e6d29e6398519a632a1  0xfe525539277edd6769133e6d29e6398519a632a1   
        0xfe91bdec3c848366415a14071d9e2b1eab800b2d  0xfe91bdec3c848366415a14071d9e2b1eab800b2d   
        0xfef2fc511d8324da83a519f00e9780475479aa62  0xfef2fc511d8324da83a519f00e9780475479aa62   
        0xff999ab0e44d4bffcabcd7b29ea99af16a5f01a4  0xff999ab0e44d4bffcabcd7b29ea99af16a5f01a4   
        0xfff2b9bb869c7886a07c3105c7ed75fa48c4ff75  0xfff2b9bb869c7886a07c3105c7ed75fa48c4ff75   

                                                              name register  \
network dao                                                                   
mainnet 0x006087d6ac20840c23ba298512db454a05c19b10     FitTogether       na   
        0x00e1b6de09e01d5b178ecf68966a34bd1dcd4064         YoyoDAO       na   
        0x00ea828bf150ec08cad91ee93841785c827b78f4       BountyDAO       na   
        0x01b46d83754ea98f3edd5e5ceb0aeb00cf291e8e     VeriDAO 1.0       na   
        0x02200bfe176e8e5f8dc58103afc8ca42ad863736          i4 Art       na   
...                                                            ...      ...   
xdai    0xfe525539277edd6769133e6d29e6398519a632a1  NiftyFamilyDao       na   
        0xfe91bdec3c848366415a14071d9e2b1eab800b2d         VISHDAO       na   
        0xfef2fc511d8324da83a519f00e9780475479aa62           C-ONE       na   
        0xff999ab0e44d4bffcabcd7b29ea99af16a5f01a4           test1       na   
        0xfff2b9bb869c7886a07c3105c7ed75fa48c4ff75      DAOrecords       na   

                                                                                   nativeToken  \
network dao                                                                                      
mainnet 0x006087d6ac20840c23ba298512db454a05c19b10  0xa3820e0f6be1c306c0a76746af80b60c228d99c2   
        0x00e1b6de09e01d5b178ecf68966a34bd1dcd4064  0xb9697151c7af8f8a4d2702c8291e3d649525b1d9   
        0x00ea828bf150ec08cad91ee93841785c827b78f4  0x41e52e9f7991513f1e0e18aa125191ce62172e5a   
        0x01b46d83754ea98f3edd5e5ceb0aeb00cf291e8e  0x8b680725307b14cbe1aea71a8d47ec41e7c05970   
        0x02200bfe176e8e5f8dc58103afc8ca42ad863736  0x425ea391f8c44b7b4abf55d74dd6de709b856c13   
...                                                                                        ...   
xdai    0xfe525539277edd6769133e6d29e6398519a632a1  0xbc2b5f764b2a2d0e446ea55e8133b42fd80ab4be   
        0xfe91bdec3c848366415a14071d9e2b1eab800b2d  0xe34dc5f9e9afca0495ea96b7cabc4d03ae083505   
        0xfef2fc511d8324da83a519f00e9780475479aa62  0x4314d7fd03c181783a6b46cc5f6cd317dcf435dd   
        0xff999ab0e44d4bffcabcd7b29ea99af16a5f01a4  0x519e98725b9779f9daf42c1211cadaebd515f658   
        0xfff2b9bb869c7886a07c3105c7ed75fa48c4ff75  0x7ba277f18bb89e8a5dba53971bc3cea510ce5d58   

                                                                              nativeReputation  \
network dao                                                                                      
mainnet 0x006087d6ac20840c23ba298512db454a05c19b10  0x63853face2fa44c4e7a8b4cf2371416c5af8e68b   
        0x00e1b6de09e01d5b178ecf68966a34bd1dcd4064  0x6950522aa12bad8976f02f331ed759b160d92727   
        0x00ea828bf150ec08cad91ee93841785c827b78f4  0xc5c1d68a7d6f2b8c290764d7a0b3ba4110b75e8c   
        0x01b46d83754ea9

# Preparing stakes df

In [5]:
dfs = pd.read_feather(DAOSTACK.STAKES)
mints = pd.read_feather(DAOSTACK.REP_MINTS)

dfs['amount'] = dfs['amount'].astype(float) / 10**18
msk = dfs['outcome'] == 'Fail'
dfs.loc[msk,'amount'] *= -1
dfs['createdAt'] = pd.to_datetime(dfs['createdAt'], unit='s')
dfs['createdAtMonth'] = dfs['createdAt'].dt.to_period('M')

# Adding the dao registered status
dfjd = dfs.merge(dfd, left_on=['network', 'dao'], right_on=['network', 'dao'])
dfs['daoRegister'] = dfjd['register']
dfs['daoName'] = dfjd['name']

# Adding internal / external
dfs['internalPlatform'] = dfs['staker'].isin(mints['address'])
def _isin(df1, df2):
    assert len(df1.columns) == len(df2.columns)
    return df1.apply(tuple, 1).isin(df2.apply(tuple, 1))

# dfs['internalDAO'] = dfs[['dao', 'staker']].apply(tuple, 1).isin(mints[['dao', 'address']].apply(tuple, 1))
dfs['internalDAO'] = _isin(dfs[['dao', 'staker']], mints[['dao', 'address']])

# Adding if the user is proposer or voter of the proposal where the stake is made
_dfp = pd.read_feather(DAOSTACK.PROPOSALS)
_dfv = pd.read_feather(DAOSTACK.VOTES)
dfs['isProposer'] = _isin(dfs[['staker', 'proposal']], _dfp[['proposer', 'id']])
dfs['isVoter'] = _isin(dfs[['staker', 'proposal']], _dfv[['voter', 'proposal']])

%pickledf dfs

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6277 entries, 0 to 6276
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   network           6277 non-null   object        
 1   id                6277 non-null   object        
 2   createdAt         6277 non-null   datetime64[ns]
 3   staker            6277 non-null   object        
 4   outcome           6277 non-null   object        
 5   amount            6277 non-null   float64       
 6   dao               6277 non-null   object        
 7   proposal          6277 non-null   object        
 8   createdAtMonth    6277 non-null   period[M]     
 9   daoRegister       6277 non-null   object        
 10  daoName           6277 non-null   object        
 11  internalPlatform  6277 non-null   bool          
 12  internalDAO       6277 non-null   bool          
 13  isProposer        6277 non-null   bool          
 14  isVoter           6277 n

,network,id,createdAt,staker,outcome,amount,dao,proposal,createdAtMonth,daoRegister,daoName,internalPlatform,internalDAO,isProposer,isVoter
0,mainnet,0x002a6f4b1986b3d700b470bdcd835a5c828e8e9cb04d...,2019-06-21 20:02:25,0x26904e77a65ba7ff95803d804ef46fc6d8f33720,Pass,60.000,0x6bee9b81e434f7afce72a43a4016719315069539,0x1d7514de86913f1f306c237d2f6c88ac31098afb00cd...,2019-06,registered,Kyber DAO Exp#2,True,True,True,True
1,mainnet,0x0034e58174fdef6f34fa3df9e4582297a74e0481d200...,2019-07-29 03:16:15,0x4abe45a46ff4be05545684ab7aa1894628d9dca6,Pass,1.000,0x519b70055af55a007110b4ff99b0ea33071c720a,0xa0225277f6b1f3b4f0bedc3e4d92ed8f3ba53e584781...,2019-07,registered,Kyber DAO Exp#2,False,False,False,False
2,mainnet,0x003855e34d7163f25f7b3abf4c8584414cf1dd3cf18c...,2019-04-22 18:52:45,0xf16294a979a027f297dace2f618cb57bc4bf5d16,Pass,150.000,0x294f999356ed03347c7a23bcbcf8d33fa41dc830,0xf37c69b6340e07c14ec9a9cac73a04682d5987d183f8...,2019-04,registered,Kyber DAO Exp#2,True,True,False,True
3,mainnet,0x004d8429b916138dfc1386b71922a9efe28c5e507eb1...,2019-04-21 14:26:21,0xce7298e5ef1ae8af0573edc2ebd03ab0f837e214,Pass,300.000,0x294f999356ed03347c7a23bcbcf8d33fa41dc830,0x9421968a79dfc6b9437c04810293ff7f732fed2190ba...,2019-04,registered,Kyber DAO Exp#2,True,True,False,True
4,mainnet,0x006812f41b36c70bc1403953407457b165f07b666d5d...,2019-08-05 19:01:08,0xa15ca74e65bf72730811abf95163e89ad9b9dff6,Fail,-100.000,0x294f999356ed03347c7a23bcbcf8d33fa41dc830,0x7fd26b2118068bd2e38e62d0b833fe49962db9e2ebaa...,2019-08,registered,Kyber DAO Exp#2,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6272,xdai,0xff8e3eaa01ced7cd2dceeba45346c3e8c8e0919e2cdd...,2022-03-18 15:54:00,0xc36cbdd85791a718cefca21045e773856a89c197,Pass,1000.000,0xe716ec63c5673b3a4732d22909b38d779fa47c3f,0x59b36d189960953e0bce203532af7f717575387eeb01...,2022-03,na,Rarible Protocol Working Group,True,True,False,True
6273,xdai,0xffdb827722dd27c4b13969d2933acb50cf4c119a1d18...,2022-06-15 16:24:05,0x1861974f32eacdcced0f81b0f8eccfed58153a9d,Pass,1500.000,0xe716ec63c5673b3a4732d22909b38d779fa47c3f,0xc7d58c37c58f7ec8557e8f6c78bf84afdc9eb14f5ec8...,2022-06,registered,BuffiDAO,True,True,False,False
6274,xdai,0xffe1f6b2cc50ac2b4b61cff94ef25edba7734d38d3e2...,2022-03-12 01:58:25,0xbc3a058d1c919f6b1f48e8846246d04d467902c8,Pass,5.000,0xe716ec63c5673b3a4732d22909b38d779fa47c3f,0x122f3766bca165927f6696f2bed943198ca8a0f2f2ef...,2022-03,registered,BuffiDAO,True,True,False,True
6275,xdai,0xffeb73bee7bb025e08acbb289ce4f7fbf1d20e79de0e...,2021-05-09 01:17:20,0x40a57a5d9b618b9a49307b530736b4f51e008741,Pass,0.010,0x94a587478c83491b13291265581cb983e7feb540,0x640e8a29be9b04dfd7fc12d11abddb0f42f312961549...,2021-05,registered,DAOstack DAO,True,True,False,True


# Preparing proposals df

In [6]:
dfp = pd.read_feather(DAOSTACK.PROPOSALS)
# dfp = dfp[dfp["stage"] == 'Executed'] # Only executed votes
_valCols = ['stakesFor', 'stakesAgainst', 'minimumDaoBounty']
dfp[_valCols] = dfp[_valCols].astype(float) / 10**18 # !! Warning, it was 2**18 previously!!
dfp['thresholdConst'] = dfp['thresholdConst'].astype(float) / 10**12
dfp = utils.append_dao_names(dfp)
dfp = utils.dfAtToDatetime(dfp)
dfp = dfp.astype({
    'network': 'category',
    'winningOutcome': 'category',
    'stage': 'category',
    'executionState': 'category',
    'queuedVoteRequiredPercentage': 'float',
    'daoBountyConst': 'int',
}).convert_dtypes()

# Convert to PYTHON int
_valCols = ['votesFor', 'votesAgainst', 'totalRepWhenCreated', 'totalRepWhenExecuted']
dfp[_valCols] = dfp[_valCols].replace({np.nan: None})
dfp[_valCols] = dfp[_valCols].applymap(lambda x: int(x) if x else None)

dfp['totalRepWhenCreated'] = dfp['totalRepWhenCreated'].where(dfp['totalRepWhenCreated'] != 0, np.NaN)

dfp['votesForPct'] = dfp['votesFor'].div(dfp['totalRepWhenCreated'])
dfp['votesAgainstPct'] = dfp['votesAgainst'].div(dfp['totalRepWhenCreated'])

dfp = dfp.set_index(['network', 'id'])

# Convert to month
dfp['executedAtMonth'] = dfp['executedAt'].dt.to_period('M')
dfp['createdAtMonth'] = dfp['createdAt'].dt.to_period('M')

# Convert to timedelta
_valCols = ['queuedVotePeriodLimit', 'boostedVotePeriodLimit']
dfp[_valCols] = dfp[_valCols].astype(int).apply(lambda col: pd.to_timedelta(col, unit='s'))

# Add more info from the DAO
## Add the dao register status
# dfjd = dfp.merge(dfd, left_on=['network', 'dao'], right_on=['network', 'dao'])
_dfd = pd.read_feather(DAOSTACK.DAOS, columns=['network', 'dao', 'register'])
dfjd = dfp.reset_index().merge(_dfd, on=['network', 'dao'], how='left').set_index(['network', 'id'])
dfp['daoRegister'] = dfjd.register.astype('category')

# Adding number of voters
VOTES_COLUMNS = ['network', 'voter', 'dao', 'proposal']
dfv = pd.read_feather(DAOSTACK.VOTES, columns=VOTES_COLUMNS)

#   Joining them. Note: Left and inner join should be the same result
dfjv = dfp.reset_index().merge(dfv, left_on=['network', 'id', 'dao'], right_on=['network', 'proposal', 'dao'], how='left', validate='one_to_many')

#   Grouping and setting the number of voters to the original proposals DataFrame
dfp['nvoters'] = dfjv.groupby(['network', 'id'])['voter'].nunique()
# print(dfp.dtypes)

# Adding the downstake created by the DAO
dfs = pd.read_feather(DAOSTACK.STAKES)
downstakes = dfs[dfs['outcome'] == 'Fail'].copy()
downstakes['amount'] = downstakes['amount'].astype(float) // 10**18
downstakes = downstakes.groupby(['network', 'proposal'])['amount'].sum()
downstakes = downstakes.reindex(dfp.index).fillna(0.0)
# warning: stakesagainst is positive
dfp['daostake'] = dfp['stakesAgainst'] - downstakes

dfp['confidence'] = dfp['stakesFor'] / dfp['stakesAgainst']
dfp['nstakers'] = dfs.groupby(['network', 'proposal'])['staker'].nunique()
dfp['nstakers'].fillna(0, inplace=True)

# Adding the self-staking
dfs = utils.get_df('dfs')
dfp['internalDAO'] = dfs.groupby(['network', 'proposal'])['internalDAO'].sum()
dfp['internalDAO'].fillna(0, inplace=True)
dfp['isProposer'] = dfs.groupby(['network', 'proposal'])['isProposer'].sum()

assert (dfp['daostake'] <= dfp['stakesAgainst']).all()
%pickledf dfp

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 5641 entries, ('mainnet', '0x0025c38d987acba1f1d446d3690384327ebe06d15f1fa4171a4dc3467f8bd416') to ('xdai', '0xffec994405570975de6a1e4fe71c4a3c5f909152fbdcd7bc734b8dd2a094aa7f')
Data columns (total 44 columns):
 #   Column                        Non-Null Count  Dtype          
---  ------                        --------------  -----          
 0   proposer                      5641 non-null   string         
 1   stage                         5641 non-null   category       
 2   createdAt                     5641 non-null   datetime64[ns] 
 3   preBoostedAt                  4026 non-null   datetime64[ns] 
 4   boostedAt                     3710 non-null   datetime64[ns] 
 5   quietEndingPeriodBeganAt      168 non-null    datetime64[ns] 
 6   closingAt                     5641 non-null   object         
 7   preBoostedClosingAt           4026 non-null   datetime64[ns] 
 8   executedAt                    5497 non-null   datetime64[ns] 
 9

proposer  \
network id                                                                                              
mainnet 0x0025c38d987acba1f1d446d3690384327ebe06d15f1f...  0xbaaea72417f4dc3e0f52a1783b0913d0f3516634   
        0x0033a337b620add50be9ba0fff1128ab866ece51cfdd...  0x052c68abe8e4bf0b78925e488b98f6fdc18a3af9   
        0x003c109411e96e5607ce609278536262e12afdaa65ab...  0xb33b9fba681653fe263b31a95766d83d18c2128d   
        0x0071c649968f7939fc9729c5dc25a8ae3f50dc86220a...  0x86fd6dd41bad636b5b3b9228bc5642fa0df392e8   
        0x007eaf0fc8d6ae35ba3f28bc7cf4b34af659db8161d4...  0x7cb9032844e722bef2e16a761832c59c96550898   
...                                                                                               ...   
xdai    0xff376c450192ebd1c911e6b8137919c80e3b291911bb...  0x152ad2e12e102abf64280c5e3d70257effb0ede0   
        0xff37f65a5cf37b109fc538ea2d18d2726f4c3c1cfdc3...  0x06044b5359d8df7886366c22c61c7ecd29becac7   
        0xff6c045d9ab8f514b7dce6317ac1ff16786cad69bd4a...  0x8e900cf9bd655e34bb610f0ef365d8d476fd7337   
        0xffc0f4a64c8ec043238ca787e58177c2e798360c2989...  0xa601aed34dda12ff760d8abb64fd4eb3664e35af   
        0xffec994405570975de6a1e4fe71c4a3c5f909152fbdc...  0x5e642f8c81de40e7e49b6a2b606dfb94a005bc44   

                                                              stage  \
network id                                                            
mainnet 0x0025c38d987acba1f1d446d3690384327ebe06d15f1f...  Executed   
        0x0033a337b620add50be9ba0fff1128ab866ece51cfdd...  Executed   
        0x003c109411e96e5607ce609278536262e12afdaa65ab...  Executed   
        0x0071c649968f7939fc9729c5dc25a8ae3f50dc86220a...  Executed   
        0x007eaf0fc8d6ae35ba3f28bc7cf4b34af659db8161d4...  Executed   
...                                                             ...   
xdai    0xff376c450192ebd1c911e6b8137919c80e3b291911bb...  Executed   
        0xff37f65a5cf37b109fc538ea2d18d2726f4c3c1cfdc3...  Executed   
        0xff6c045d9ab8f514b7dce6317ac1ff16786cad69bd4a...  Executed   
        0xffc0f4a64c8ec043238ca787e58177c2e798360c2989...  Executed   
        0xffec994405570975de6a1e4fe71c4a3c5f909152fbdc...  Executed   

                                                                    createdAt  \
network id                                                                      
mainnet 0x0025c38d987acba1f1d446d3690384327ebe06d15f1f... 2019-10-22 10:07:58   
        0x0033a337b620add50be9ba0fff1128ab866ece51cfdd... 2020-01-17 19:07:19   
        0x003c109411e96e5607ce609278536262e12afdaa65ab... 2020-03-14 17:52:32   
        0x0071c649968f7939fc9729c5dc25a8ae3f50dc86220a... 2019-12-14 14:52:54   
        0x007eaf0fc8d6ae35ba3f28bc7cf4b34af659db8161d4... 2019-11-01 19:26:29   
...                                                                       ...   
xdai    0xff376c450192ebd1c911e6b8137919c80e3b291911bb... 2021-04-20 23:40:15   
        0xff37f65a5cf37b109fc538ea2d18d2726f4c3c1cfdc3... 2020-02-11 13:36:25   
        0xff6c045d9ab8f514b7dce6317ac1ff16786cad69bd4a... 2022-02-08 09:23:30   
        0xffc0f4a64c8ec043238ca787e58177c2e798360c2989... 2021-01-22 19:09:15   
        0xffec994405570975de6a1e4fe71c4a3c5f909152fbdc... 2021-05-03 22:01:50   

                                                                 preBoostedAt  \
network id                                                                      
mainnet 0x0025c38d987acba1f1d446d3690384327ebe06d15f1f... 2019-10-22 10:39:28   
        0x0033a337b620add50be9ba0fff1128ab866ece51cfdd...                 NaT   
        0x003c109411e96e5607ce609278536262e12afdaa65ab... 2020-03-14 19:53:49   
        0x0071c649968f7939fc9729c5dc25a8ae3f50dc86220a... 2019-12-14 18:13:24   
        0x007eaf0fc8d6ae35ba3f28bc7cf4b34af659db8161d4... 2019-11-01 19:34:07   
...                                                                       ...   
xdai    0xff376c450192ebd1c911e6b8137919c80e3b291911bb... 2021-04-21 16:18:15   
        0xff37f65a5c

In [7]:
dfp['stage'].value_counts()

Executed          4824
ExpiredInQueue     673
Queued             137
Boosted              6
PreBoosted           1
Name: stage, dtype: int64

In [8]:
dfp[['stage', 'executionState']].value_counts()

stage           executionState      
Executed        BoostedTimeOut          3476
                QueueBarCrossed          940
ExpiredInQueue  QueueTimeOut             673
Executed        BoostedBarCrossed        228
                PreBoostedBarCrossed     180
Queued          None                     137
Boosted         None                       6
PreBoosted      None                       1
dtype: int64

In [9]:
dfp[dfp.stage == 'Boosted']

proposer  \
network id                                                                                              
mainnet 0x02c0c0f653e29351e3eb40a67df85b1e844da883c350...  0x639749b7b08aee65039c21d8a411103c6cebebf0   
        0x21cc40c5587fec1dd138391849be93fcb6decd647ecd...  0xb1b7586656116d546033e3baff69bfcd6592225e   
        0x84e855e489ac88923ec0d130656566109c79fa23b0f1...  0xb1b7586656116d546033e3baff69bfcd6592225e   
        0x9db177024f122afa9bafe4df77f0750172b0d6a7abeb...  0x91628ddc3a6ff9b48a2f34fc315d243eb07a9501   
xdai    0x08e78c6aa36633cbf48e2fa5d00332980da6886e3833...  0x08eec580ad41e9994599bad7d2a74a9874a2852c   
        0x8ec7d23b51bc4479adf1d27967236f57277eca61cf7e...  0x08eec580ad41e9994599bad7d2a74a9874a2852c   

                                                             stage  \
network id                                                           
mainnet 0x02c0c0f653e29351e3eb40a67df85b1e844da883c350...  Boosted   
        0x21cc40c5587fec1dd138391849be93fcb6decd647ecd...  Boosted   
        0x84e855e489ac88923ec0d130656566109c79fa23b0f1...  Boosted   
        0x9db177024f122afa9bafe4df77f0750172b0d6a7abeb...  Boosted   
xdai    0x08e78c6aa36633cbf48e2fa5d00332980da6886e3833...  Boosted   
        0x8ec7d23b51bc4479adf1d27967236f57277eca61cf7e...  Boosted   

                                                                    createdAt  \
network id                                                                      
mainnet 0x02c0c0f653e29351e3eb40a67df85b1e844da883c350... 2021-07-25 22:51:15   
        0x21cc40c5587fec1dd138391849be93fcb6decd647ecd... 2019-07-12 20:10:50   
        0x84e855e489ac88923ec0d130656566109c79fa23b0f1... 2019-07-12 20:10:30   
        0x9db177024f122afa9bafe4df77f0750172b0d6a7abeb... 2023-03-21 01:39:59   
xdai    0x08e78c6aa36633cbf48e2fa5d00332980da6886e3833... 2022-05-30 14:47:00   
        0x8ec7d23b51bc4479adf1d27967236f57277eca61cf7e... 2022-05-30 14:42:40   

                                                                 preBoostedAt  \
network id                                                                      
mainnet 0x02c0c0f653e29351e3eb40a67df85b1e844da883c350... 2021-07-25 22:55:38   
        0x21cc40c5587fec1dd138391849be93fcb6decd647ecd... 2019-07-13 17:08:05   
        0x84e855e489ac88923ec0d130656566109c79fa23b0f1... 2019-07-13 17:08:05   
        0x9db177024f122afa9bafe4df77f0750172b0d6a7abeb... 2023-03-21 02:36:47   
xdai    0x08e78c6aa36633cbf48e2fa5d00332980da6886e3833... 2022-06-07 18:32:25   
        0x8ec7d23b51bc4479adf1d27967236f57277eca61cf7e... 2022-06-07 18:28:25   

                                                                    boostedAt  \
network id                                                                      
mainnet 0x02c0c0f653e29351e3eb40a67df85b1e844da883c350... 2021-07-26 11:57:45   
        0x21cc40c5587fec1dd138391849be93fcb6decd647ecd... 2019-07-14 05:08:19   
        0x84e855e489ac88923ec0d130656566109c79fa23b0f1... 2019-07-14 09:15:02   
        0x9db177024f122afa9bafe4df77f0750172b0d6a7abeb... 2023-03-24 15:59:47   
xdai    0x08e78c6aa36633cbf48e2fa5d00332980da6886e3833... 2022-06-08 23:42:40   
        0x8ec7d23b51bc4479adf1d27967236f57277eca61cf7e... 2022-06-08 23:41:55   

                                                          quietEndingPeriodBeganAt  \
network id                                                                           
mainnet 0x02c0c0f653e29351e3eb40a67df85b1e844da883c350...                      NaT   
        0x21cc40c5587fec1dd138391849be93fcb6decd647ecd...                      NaT   
        0x84e855e489ac88923ec0d130656566109c79fa23b0f1...                      NaT   
        0x9db177024f122afa9bafe4df77f0750172b0d6a7abeb...                      NaT   
xdai    0x08e78c6aa36633cbf48e2fa5d00332980da6886e3833...                      NaT   
        0x8ec7d23b51bc4479adf1d27967236f57277eca61cf7e...                      NaT   

                                                       

In [10]:
dfp['closingAt'].isna().sum()

0

In [11]:
# Checking that all get_df works fine
aux = utils.get_df('dfp', registered_only=True)
assert aux[(aux.daoRegister != 'registered') & (aux.daoRegister != 'unRegistered')].empty, "Inconsistence error"

# Preparing queue df

In [12]:
print("at columns:", ", ".join(c for c in dfp.columns if c.lower().endswith("at")))

def getPropQ(row: pd.Series):
    """ gets the queue event times from the given proposal 
        Returns: a tuple (queued, unqueued) where queued is the time it was queued, and
        unqueued can be NaT if it is still in the queue
    """  
    queued = row['createdAt']
    # unqueued = pd.Timestamp.max
    unqueued = pd.NaT
    reason = None
    
    # for c in ['executedAt', 'preBoostedAt', 'expiresInQueueAt']:
    for c in ['boostedAt', 'executedAt', 'expiresInQueueAt']:
        # if row[c] < unqueued:
        if pd.isna(unqueued) and pd.notna(row[c]):
            unqueued = row[c]
            reason = c
            break
    
    return pd.Series(data={
        'queued': queued,
        'unqueued': unqueued,
        'reason': reason,
        'dao': row['dao'],
        'daoRegister': row['daoRegister'],
        'daoName': row['name'],
    }, name=row.name)

dfq = dfp.progress_apply(getPropQ, axis='columns')
assert (dfq.unqueued.isna() | (dfq.queued <= dfq.unqueued)).all()
dfq['delta'] = dfq['unqueued'] - dfq['queued']

# Removing the ones with delta 0 because they don't affect queue size
# dfq = dfq[dfq['queued'] != dfq['unqueued']]
%pickledf dfq

at columns: createdAt, preBoostedAt, boostedAt, quietEndingPeriodBeganAt, closingAt, preBoostedClosingAt, executedAt, expiresInQueueAt


  0%|          | 0/5641 [00:00<?, ?it/s]

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 5641 entries, ('mainnet', '0x0025c38d987acba1f1d446d3690384327ebe06d15f1fa4171a4dc3467f8bd416') to ('xdai', '0xffec994405570975de6a1e4fe71c4a3c5f909152fbdcd7bc734b8dd2a094aa7f')
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype          
---  ------       --------------  -----          
 0   queued       5641 non-null   datetime64[ns] 
 1   unqueued     5641 non-null   datetime64[ns] 
 2   reason       5641 non-null   object         
 3   dao          5641 non-null   object         
 4   daoRegister  5641 non-null   object         
 5   daoName      3967 non-null   object         
 6   delta        5641 non-null   timedelta64[ns]
dtypes: datetime64[ns](2), object(4), timedelta64[ns](1)
memory usage: 498.4+ KB
None
dfq pickled to .pickles/dfq.pickle


queued  \
network id                                                                      
mainnet 0x0025c38d987acba1f1d446d3690384327ebe06d15f1f... 2019-10-22 10:07:58   
        0x0033a337b620add50be9ba0fff1128ab866ece51cfdd... 2020-01-17 19:07:19   
        0x003c109411e96e5607ce609278536262e12afdaa65ab... 2020-03-14 17:52:32   
        0x0071c649968f7939fc9729c5dc25a8ae3f50dc86220a... 2019-12-14 14:52:54   
        0x007eaf0fc8d6ae35ba3f28bc7cf4b34af659db8161d4... 2019-11-01 19:26:29   
...                                                                       ...   
xdai    0xff376c450192ebd1c911e6b8137919c80e3b291911bb... 2021-04-20 23:40:15   
        0xff37f65a5cf37b109fc538ea2d18d2726f4c3c1cfdc3... 2020-02-11 13:36:25   
        0xff6c045d9ab8f514b7dce6317ac1ff16786cad69bd4a... 2022-02-08 09:23:30   
        0xffc0f4a64c8ec043238ca787e58177c2e798360c2989... 2021-01-22 19:09:15   
        0xffec994405570975de6a1e4fe71c4a3c5f909152fbdc... 2021-05-03 22:01:50   

                                                                     unqueued  \
network id                                                                      
mainnet 0x0025c38d987acba1f1d446d3690384327ebe06d15f1f... 2019-10-23 12:44:17   
        0x0033a337b620add50be9ba0fff1128ab866ece51cfdd... 2020-01-17 19:39:16   
        0x003c109411e96e5607ce609278536262e12afdaa65ab... 2020-03-15 19:54:03   
        0x0071c649968f7939fc9729c5dc25a8ae3f50dc86220a... 2019-12-15 21:57:46   
        0x007eaf0fc8d6ae35ba3f28bc7cf4b34af659db8161d4... 2019-11-03 07:48:08   
...                                                                       ...   
xdai    0xff376c450192ebd1c911e6b8137919c80e3b291911bb... 2021-04-22 16:26:50   
        0xff37f65a5cf37b109fc538ea2d18d2726f4c3c1cfdc3... 2020-02-11 13:37:30   
        0xff6c045d9ab8f514b7dce6317ac1ff16786cad69bd4a... 2022-02-09 10:51:15   
        0xffc0f4a64c8ec043238ca787e58177c2e798360c2989... 2021-01-23 21:10:55   
        0xffec994405570975de6a1e4fe71c4a3c5f909152fbdc... 2021-05-07 03:20:10   

                                                               reason  \
network id                                                              
mainnet 0x0025c38d987acba1f1d446d3690384327ebe06d15f1f...   boostedAt   
        0x0033a337b620add50be9ba0fff1128ab866ece51cfdd...  executedAt   
        0x003c109411e96e5607ce609278536262e12afdaa65ab...   boostedAt   
        0x0071c649968f7939fc9729c5dc25a8ae3f50dc86220a...   boostedAt   
        0x007eaf0fc8d6ae35ba3f28bc7cf4b34af659db8161d4...   boostedAt   
...                                                               ...   
xdai    0xff376c450192ebd1c911e6b8137919c80e3b291911bb...   boostedAt   
        0xff37f65a5cf37b109fc538ea2d18d2726f4c3c1cfdc3...  executedAt   
        0xff6c045d9ab8f514b7dce6317ac1ff16786cad69bd4a...   boostedAt   
        0xffc0f4a64c8ec043238ca787e58177c2e798360c2989...   boostedAt   
        0xffec994405570975de6a1e4fe71c4a3c5f909152fbdc...   boostedAt   

                                                                                                  dao  \
network id                                                                                              
mainnet 0x0025c38d987acba1f1d446d3690384327ebe06d15f1f...  0x294f999356ed03347c7a23bcbcf8d33fa41dc830   
        0x0033a337b620add50be9ba0fff1128ab866ece51cfdd...  0x39757e7cdb7022c3829195d57b29428d79593c9d   
        0x003c109411e96e5607ce609278536262e12afdaa65ab...  0x519b70055af55a007110b4ff99b0ea33071c720a   
        0x0071c649968f7939fc9729c5dc25a8ae3f50dc86220a...  0x8990e11b69403ea53ef2b32434bbd7dbf84b5234   
        0x007eaf0fc8d6ae35ba3f28bc7cf4b34af659db8161d4...  0x440583455bcd85ab2bd429c015d3aabcae135f0a   
...                                                                                               ...   
xdai    0xff376c450192ebd1c911e6b8137919c80e3b291911bb...  0x94a587478c83491b13291265581cb983e7feb540   
        0xff37f65a5cf37b109fc538ea2d18d2726f4c3c1cfdc3...  0x4875e10

In [13]:
dfq.reason.value_counts()

boostedAt           3710
executedAt          1793
expiresInQueueAt     138
Name: reason, dtype: int64

## Preparing queue count df (dfqcnt)

In [14]:
cols = ['at', 'network', 'id', 'dao', 'daoRegister', 'daoName']

# TODO: Shouldn't the index be date, network, id
# And if only date, network is needed, the cnt should be the sum of internal cnt
dfqueued = dfq.reset_index().rename(columns={'queued': 'at'})[cols]
dfunqueued = dfq.dropna(subset='unqueued').reset_index().rename(columns={'unqueued': 'at'})[cols]
dfqueued['cnt'] = 1
dfunqueued['cnt'] = -1

dfqcnt = pd.concat([dfqueued, dfunqueued]).sort_values('at')
%pickledf dfqcnt

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11282 entries, 1158 to 2325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   at           11282 non-null  datetime64[ns]
 1   network      11282 non-null  category      
 2   id           11282 non-null  string        
 3   dao          11282 non-null  object        
 4   daoRegister  11282 non-null  object        
 5   daoName      7934 non-null   object        
 6   cnt          11282 non-null  int64         
dtypes: category(1), datetime64[ns](1), int64(1), object(3), string(1)
memory usage: 628.1+ KB
None
dfqcnt pickled to .pickles/dfqcnt.pickle


,at,network,id,dao,daoRegister,daoName,cnt
1158,2019-04-10 18:59:04,mainnet,0x748b23205728476ea4d3e9d3a752bc1f7fdaadf660e9...,0x294f999356ed03347c7a23bcbcf8d33fa41dc830,registered,Genesis Alpha,1
1841,2019-04-10 18:59:11,mainnet,0xb775ceea172980b5130db7eb24d07fe9713338845c1b...,0x294f999356ed03347c7a23bcbcf8d33fa41dc830,registered,Genesis Alpha,1
2407,2019-04-10 19:01:47,mainnet,0xf038967ac6e37121f54d5d4168df9d717282d63fac3f...,0x294f999356ed03347c7a23bcbcf8d33fa41dc830,registered,Genesis Alpha,1
2148,2019-04-11 06:42:42,mainnet,0xd709125c86ef454d72519f6ae6551fe974fa56e27489...,0x294f999356ed03347c7a23bcbcf8d33fa41dc830,registered,Genesis Alpha,1
915,2019-04-11 06:47:10,mainnet,0x5b623c615c5713b9a1bcd41cf73687d11c37c9d7b2af...,0x294f999356ed03347c7a23bcbcf8d33fa41dc830,registered,Genesis Alpha,1
...,...,...,...,...,...,...,...
128,2023-04-05 13:21:11,mainnet,0x0dc340bf6583557c718716d50d6fc8adbc6f73ab7414...,0x519b70055af55a007110b4ff99b0ea33071c720a,registered,dxDAO,1
2991,2023-04-05 13:25:00,xdai,0x2595e6bff6eb185088b709bd1c9a7058919ca22caed6...,0xe716ec63c5673b3a4732d22909b38d779fa47c3f,registered,xDXdao,1
2991,2023-04-07 09:34:55,xdai,0x2595e6bff6eb185088b709bd1c9a7058919ca22caed6...,0xe716ec63c5673b3a4732d22909b38d779fa47c3f,registered,xDXdao,-1
128,2023-04-07 15:36:11,mainnet,0x0dc340bf6583557c718716d50d6fc8adbc6f73ab7414...,0x519b70055af55a007110b4ff99b0ea33071c720a,registered,dxDAO,-1


# Preparing open proposals df

Una propuesta está abierta (open), desde que se crea hasta que se ejecuta o caduca en la cola.

Ignoramos el campo _closingAt_, pues indica cuando se cierra el proceso actual, y no indica cuando se cierra la propuesta en sí.

In [15]:
print("at columns:", ", ".join(c for c in dfp.columns if c.lower().endswith("at")))

at columns: createdAt, preBoostedAt, boostedAt, quietEndingPeriodBeganAt, closingAt, preBoostedClosingAt, executedAt, expiresInQueueAt


In [16]:
def getPropO(row: pd.Series):
    """ gets the queue event times from the given proposal 
        Returns: a tuple (queued, unqueued) where queued is the time it was queued, and
        unqueued can be NaT if it is still in the queue
    """  
    opened = row['createdAt']
    # closed = pd.Timestamp.min
    closed = pd.NaT
    reason = None
    
    # Warning: executedAt should take precedence
    
    match row['stage'].lower():
        case "boosted" | "quietendingperiod":
            cols = ['closingAt', 'preBoostedClosingAt']
        case _:
            cols = ['executedAt', 'expiresInQueueAt']
            
    for c in cols:
        # if row[c] > closed:
        if pd.isna(closed) and pd.notna(row[c]):
            closed = row[c]
            reason = c
            break
            
    if closed < opened:
        reason += 'Error'
    
    return pd.Series(data={
        'opened': opened,
        'closed': closed,
        'reason': reason,
        'dao': row['dao'],
        'daoRegister': row['daoRegister'],
        'daoName': row['name'],
    }, name=row.name)

dfo = dfp.progress_apply(getPropO, axis='columns')
assert (dfo['closed'].isna() | (dfo['opened'] <= dfo['closed'])).all(), "Requests should be open before closing"
dfo['delta'] = dfo['closed'] - dfo['opened']

# Removing the ones with delta 0 because they don't affect queue size
# dfo = dfo[dfo['opened'] != dfo['closed']]
%pickledf dfo

  0%|          | 0/5641 [00:00<?, ?it/s]

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 5641 entries, ('mainnet', '0x0025c38d987acba1f1d446d3690384327ebe06d15f1fa4171a4dc3467f8bd416') to ('xdai', '0xffec994405570975de6a1e4fe71c4a3c5f909152fbdcd7bc734b8dd2a094aa7f')
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype          
---  ------       --------------  -----          
 0   opened       5641 non-null   datetime64[ns] 
 1   closed       5641 non-null   datetime64[ns] 
 2   reason       5641 non-null   object         
 3   dao          5641 non-null   object         
 4   daoRegister  5641 non-null   object         
 5   daoName      3967 non-null   object         
 6   delta        5641 non-null   timedelta64[ns]
dtypes: datetime64[ns](2), object(4), timedelta64[ns](1)
memory usage: 498.4+ KB
None
dfo pickled to .pickles/dfo.pickle


opened  \
network id                                                                      
mainnet 0x0025c38d987acba1f1d446d3690384327ebe06d15f1f... 2019-10-22 10:07:58   
        0x0033a337b620add50be9ba0fff1128ab866ece51cfdd... 2020-01-17 19:07:19   
        0x003c109411e96e5607ce609278536262e12afdaa65ab... 2020-03-14 17:52:32   
        0x0071c649968f7939fc9729c5dc25a8ae3f50dc86220a... 2019-12-14 14:52:54   
        0x007eaf0fc8d6ae35ba3f28bc7cf4b34af659db8161d4... 2019-11-01 19:26:29   
...                                                                       ...   
xdai    0xff376c450192ebd1c911e6b8137919c80e3b291911bb... 2021-04-20 23:40:15   
        0xff37f65a5cf37b109fc538ea2d18d2726f4c3c1cfdc3... 2020-02-11 13:36:25   
        0xff6c045d9ab8f514b7dce6317ac1ff16786cad69bd4a... 2022-02-08 09:23:30   
        0xffc0f4a64c8ec043238ca787e58177c2e798360c2989... 2021-01-22 19:09:15   
        0xffec994405570975de6a1e4fe71c4a3c5f909152fbdc... 2021-05-03 22:01:50   

                                                                       closed  \
network id                                                                      
mainnet 0x0025c38d987acba1f1d446d3690384327ebe06d15f1f... 2019-10-27 12:44:18   
        0x0033a337b620add50be9ba0fff1128ab866ece51cfdd... 2020-01-17 19:39:16   
        0x003c109411e96e5607ce609278536262e12afdaa65ab... 2020-03-22 19:54:21   
        0x0071c649968f7939fc9729c5dc25a8ae3f50dc86220a... 2019-12-18 21:57:59   
        0x007eaf0fc8d6ae35ba3f28bc7cf4b34af659db8161d4... 2019-11-06 09:25:53   
...                                                                       ...   
xdai    0xff376c450192ebd1c911e6b8137919c80e3b291911bb... 2021-04-30 19:17:55   
        0xff37f65a5cf37b109fc538ea2d18d2726f4c3c1cfdc3... 2020-02-11 13:37:30   
        0xff6c045d9ab8f514b7dce6317ac1ff16786cad69bd4a... 2022-02-13 10:51:30   
        0xffc0f4a64c8ec043238ca787e58177c2e798360c2989... 2021-01-27 21:11:00   
        0xffec994405570975de6a1e4fe71c4a3c5f909152fbdc... 2021-05-11 03:20:15   

                                                               reason  \
network id                                                              
mainnet 0x0025c38d987acba1f1d446d3690384327ebe06d15f1f...  executedAt   
        0x0033a337b620add50be9ba0fff1128ab866ece51cfdd...  executedAt   
        0x003c109411e96e5607ce609278536262e12afdaa65ab...  executedAt   
        0x0071c649968f7939fc9729c5dc25a8ae3f50dc86220a...  executedAt   
        0x007eaf0fc8d6ae35ba3f28bc7cf4b34af659db8161d4...  executedAt   
...                                                               ...   
xdai    0xff376c450192ebd1c911e6b8137919c80e3b291911bb...  executedAt   
        0xff37f65a5cf37b109fc538ea2d18d2726f4c3c1cfdc3...  executedAt   
        0xff6c045d9ab8f514b7dce6317ac1ff16786cad69bd4a...  executedAt   
        0xffc0f4a64c8ec043238ca787e58177c2e798360c2989...  executedAt   
        0xffec994405570975de6a1e4fe71c4a3c5f909152fbdc...  executedAt   

                                                                                                  dao  \
network id                                                                                              
mainnet 0x0025c38d987acba1f1d446d3690384327ebe06d15f1f...  0x294f999356ed03347c7a23bcbcf8d33fa41dc830   
        0x0033a337b620add50be9ba0fff1128ab866ece51cfdd...  0x39757e7cdb7022c3829195d57b29428d79593c9d   
        0x003c109411e96e5607ce609278536262e12afdaa65ab...  0x519b70055af55a007110b4ff99b0ea33071c720a   
        0x0071c649968f7939fc9729c5dc25a8ae3f50dc86220a...  0x8990e11b69403ea53ef2b32434bbd7dbf84b5234   
        0x007eaf0fc8d6ae35ba3f28bc7cf4b34af659db8161d4...  0x440583455bcd85ab2bd429c015d3aabcae135f0a   
...                                                                                               ...   
xdai    0xff376c450192ebd1c911e6b8137919c80e3b291911bb...  0x94a587478c83491b13291265581cb983e7feb540   
        0xff37f65a5cf37b109fc538ea2d18d2726f4c3c1cfdc3...  0x4875e10

In [17]:
dfo.reason.value_counts()

executedAt          5497
expiresInQueueAt     138
closingAt              6
Name: reason, dtype: int64

In [18]:
[x for x in dfp.columns if x.endswith('At')]

['createdAt',
 'preBoostedAt',
 'boostedAt',
 'quietEndingPeriodBeganAt',
 'closingAt',
 'preBoostedClosingAt',
 'executedAt',
 'expiresInQueueAt']

In [19]:
_aux = dfp.drop(columns=['totalRepWhenExecuted', 'totalRepWhenCreated', 'votesFor', 'votesAgainst', 'winningOutcome', 'stakesFor', 'stakesAgainst', 'queuedVoteRequiredPercentage', 'minimumDaoBounty'])
_aux[_aux['stage'] == 'Boosted']

proposer  \
network id                                                                                              
mainnet 0x02c0c0f653e29351e3eb40a67df85b1e844da883c350...  0x639749b7b08aee65039c21d8a411103c6cebebf0   
        0x21cc40c5587fec1dd138391849be93fcb6decd647ecd...  0xb1b7586656116d546033e3baff69bfcd6592225e   
        0x84e855e489ac88923ec0d130656566109c79fa23b0f1...  0xb1b7586656116d546033e3baff69bfcd6592225e   
        0x9db177024f122afa9bafe4df77f0750172b0d6a7abeb...  0x91628ddc3a6ff9b48a2f34fc315d243eb07a9501   
xdai    0x08e78c6aa36633cbf48e2fa5d00332980da6886e3833...  0x08eec580ad41e9994599bad7d2a74a9874a2852c   
        0x8ec7d23b51bc4479adf1d27967236f57277eca61cf7e...  0x08eec580ad41e9994599bad7d2a74a9874a2852c   

                                                             stage  \
network id                                                           
mainnet 0x02c0c0f653e29351e3eb40a67df85b1e844da883c350...  Boosted   
        0x21cc40c5587fec1dd138391849be93fcb6decd647ecd...  Boosted   
        0x84e855e489ac88923ec0d130656566109c79fa23b0f1...  Boosted   
        0x9db177024f122afa9bafe4df77f0750172b0d6a7abeb...  Boosted   
xdai    0x08e78c6aa36633cbf48e2fa5d00332980da6886e3833...  Boosted   
        0x8ec7d23b51bc4479adf1d27967236f57277eca61cf7e...  Boosted   

                                                                    createdAt  \
network id                                                                      
mainnet 0x02c0c0f653e29351e3eb40a67df85b1e844da883c350... 2021-07-25 22:51:15   
        0x21cc40c5587fec1dd138391849be93fcb6decd647ecd... 2019-07-12 20:10:50   
        0x84e855e489ac88923ec0d130656566109c79fa23b0f1... 2019-07-12 20:10:30   
        0x9db177024f122afa9bafe4df77f0750172b0d6a7abeb... 2023-03-21 01:39:59   
xdai    0x08e78c6aa36633cbf48e2fa5d00332980da6886e3833... 2022-05-30 14:47:00   
        0x8ec7d23b51bc4479adf1d27967236f57277eca61cf7e... 2022-05-30 14:42:40   

                                                                 preBoostedAt  \
network id                                                                      
mainnet 0x02c0c0f653e29351e3eb40a67df85b1e844da883c350... 2021-07-25 22:55:38   
        0x21cc40c5587fec1dd138391849be93fcb6decd647ecd... 2019-07-13 17:08:05   
        0x84e855e489ac88923ec0d130656566109c79fa23b0f1... 2019-07-13 17:08:05   
        0x9db177024f122afa9bafe4df77f0750172b0d6a7abeb... 2023-03-21 02:36:47   
xdai    0x08e78c6aa36633cbf48e2fa5d00332980da6886e3833... 2022-06-07 18:32:25   
        0x8ec7d23b51bc4479adf1d27967236f57277eca61cf7e... 2022-06-07 18:28:25   

                                                                    boostedAt  \
network id                                                                      
mainnet 0x02c0c0f653e29351e3eb40a67df85b1e844da883c350... 2021-07-26 11:57:45   
        0x21cc40c5587fec1dd138391849be93fcb6decd647ecd... 2019-07-14 05:08:19   
        0x84e855e489ac88923ec0d130656566109c79fa23b0f1... 2019-07-14 09:15:02   
        0x9db177024f122afa9bafe4df77f0750172b0d6a7abeb... 2023-03-24 15:59:47   
xdai    0x08e78c6aa36633cbf48e2fa5d00332980da6886e3833... 2022-06-08 23:42:40   
        0x8ec7d23b51bc4479adf1d27967236f57277eca61cf7e... 2022-06-08 23:41:55   

                                                          quietEndingPeriodBeganAt  \
network id                                                                           
mainnet 0x02c0c0f653e29351e3eb40a67df85b1e844da883c350...                      NaT   
        0x21cc40c5587fec1dd138391849be93fcb6decd647ecd...                      NaT   
        0x84e855e489ac88923ec0d130656566109c79fa23b0f1...                      NaT   
        0x9db177024f122afa9bafe4df77f0750172b0d6a7abeb...                      NaT   
xdai    0x08e78c6aa36633cbf48e2fa5d00332980da6886e3833...                      NaT   
        0x8ec7d23b51bc4479adf1d27967236f57277eca61cf7e...                      NaT   

                                                       

## Preparing open count df (dfocnt)

In [20]:
cols = ['at', 'network', 'id', 'dao', 'daoRegister', 'daoName']

# TODO: Shouldn't the index be date, network, id
# And if only date, network is needed, the cnt should be the sum of internal cnt
dfopened = dfo.reset_index().rename(columns={'opened': 'at'})[cols]
dfclosed = dfo.dropna(subset='closed').reset_index().rename(columns={'closed': 'at'})[cols]
dfopened['cnt'] = 1
dfclosed['cnt'] = -1

dfocnt = pd.concat([dfopened, dfclosed]).sort_values('at')
%pickledf dfocnt

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11282 entries, 1158 to 2325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   at           11282 non-null  datetime64[ns]
 1   network      11282 non-null  category      
 2   id           11282 non-null  string        
 3   dao          11282 non-null  object        
 4   daoRegister  11282 non-null  object        
 5   daoName      7934 non-null   object        
 6   cnt          11282 non-null  int64         
dtypes: category(1), datetime64[ns](1), int64(1), object(3), string(1)
memory usage: 628.1+ KB
None
dfocnt pickled to .pickles/dfocnt.pickle


,at,network,id,dao,daoRegister,daoName,cnt
1158,2019-04-10 18:59:04,mainnet,0x748b23205728476ea4d3e9d3a752bc1f7fdaadf660e9...,0x294f999356ed03347c7a23bcbcf8d33fa41dc830,registered,Genesis Alpha,1
1841,2019-04-10 18:59:11,mainnet,0xb775ceea172980b5130db7eb24d07fe9713338845c1b...,0x294f999356ed03347c7a23bcbcf8d33fa41dc830,registered,Genesis Alpha,1
2407,2019-04-10 19:01:47,mainnet,0xf038967ac6e37121f54d5d4168df9d717282d63fac3f...,0x294f999356ed03347c7a23bcbcf8d33fa41dc830,registered,Genesis Alpha,1
2148,2019-04-11 06:42:42,mainnet,0xd709125c86ef454d72519f6ae6551fe974fa56e27489...,0x294f999356ed03347c7a23bcbcf8d33fa41dc830,registered,Genesis Alpha,1
915,2019-04-11 06:47:10,mainnet,0x5b623c615c5713b9a1bcd41cf73687d11c37c9d7b2af...,0x294f999356ed03347c7a23bcbcf8d33fa41dc830,registered,Genesis Alpha,1
...,...,...,...,...,...,...,...
1585,2023-04-07 17:59:47,mainnet,0x9db177024f122afa9bafe4df77f0750172b0d6a7abeb...,0x519b70055af55a007110b4ff99b0ea33071c720a,registered,dxDAO,-1
447,2023-04-08 13:37:47,mainnet,0x2f701be3fbd3e7e706f1aac4d36839c18a56a52e45e5...,0x519b70055af55a007110b4ff99b0ea33071c720a,registered,dxDAO,-1
128,2023-04-09 11:34:47,mainnet,0x0dc340bf6583557c718716d50d6fc8adbc6f73ab7414...,0x519b70055af55a007110b4ff99b0ea33071c720a,registered,dxDAO,-1
2991,2023-04-11 09:35:15,xdai,0x2595e6bff6eb185088b709bd1c9a7058919ca22caed6...,0xe716ec63c5673b3a4732d22909b38d779fa47c3f,registered,xDXdao,-1


## Check open and queue integrity

NO debería haber propuestas cerradas pero aún en la cola

In [21]:
dfqo = dfq.join(dfo, lsuffix='_q', rsuffix='_o')

assert (dfqo.queued == dfqo.opened).all(), "Opened and queued should be the same"
assertMsk = dfqo.closed >= dfqo.unqueued
if not assertMsk.all(): display(dfqo[~assertMsk])
assert assertMsk.all(), "Every closed proposal should be unqueued before or at the same time"
dfqo

queued  \
network id                                                                      
mainnet 0x0025c38d987acba1f1d446d3690384327ebe06d15f1f... 2019-10-22 10:07:58   
        0x0033a337b620add50be9ba0fff1128ab866ece51cfdd... 2020-01-17 19:07:19   
        0x003c109411e96e5607ce609278536262e12afdaa65ab... 2020-03-14 17:52:32   
        0x0071c649968f7939fc9729c5dc25a8ae3f50dc86220a... 2019-12-14 14:52:54   
        0x007eaf0fc8d6ae35ba3f28bc7cf4b34af659db8161d4... 2019-11-01 19:26:29   
...                                                                       ...   
xdai    0xff376c450192ebd1c911e6b8137919c80e3b291911bb... 2021-04-20 23:40:15   
        0xff37f65a5cf37b109fc538ea2d18d2726f4c3c1cfdc3... 2020-02-11 13:36:25   
        0xff6c045d9ab8f514b7dce6317ac1ff16786cad69bd4a... 2022-02-08 09:23:30   
        0xffc0f4a64c8ec043238ca787e58177c2e798360c2989... 2021-01-22 19:09:15   
        0xffec994405570975de6a1e4fe71c4a3c5f909152fbdc... 2021-05-03 22:01:50   

                                                                     unqueued  \
network id                                                                      
mainnet 0x0025c38d987acba1f1d446d3690384327ebe06d15f1f... 2019-10-23 12:44:17   
        0x0033a337b620add50be9ba0fff1128ab866ece51cfdd... 2020-01-17 19:39:16   
        0x003c109411e96e5607ce609278536262e12afdaa65ab... 2020-03-15 19:54:03   
        0x0071c649968f7939fc9729c5dc25a8ae3f50dc86220a... 2019-12-15 21:57:46   
        0x007eaf0fc8d6ae35ba3f28bc7cf4b34af659db8161d4... 2019-11-03 07:48:08   
...                                                                       ...   
xdai    0xff376c450192ebd1c911e6b8137919c80e3b291911bb... 2021-04-22 16:26:50   
        0xff37f65a5cf37b109fc538ea2d18d2726f4c3c1cfdc3... 2020-02-11 13:37:30   
        0xff6c045d9ab8f514b7dce6317ac1ff16786cad69bd4a... 2022-02-09 10:51:15   
        0xffc0f4a64c8ec043238ca787e58177c2e798360c2989... 2021-01-23 21:10:55   
        0xffec994405570975de6a1e4fe71c4a3c5f909152fbdc... 2021-05-07 03:20:10   

                                                             reason_q  \
network id                                                              
mainnet 0x0025c38d987acba1f1d446d3690384327ebe06d15f1f...   boostedAt   
        0x0033a337b620add50be9ba0fff1128ab866ece51cfdd...  executedAt   
        0x003c109411e96e5607ce609278536262e12afdaa65ab...   boostedAt   
        0x0071c649968f7939fc9729c5dc25a8ae3f50dc86220a...   boostedAt   
        0x007eaf0fc8d6ae35ba3f28bc7cf4b34af659db8161d4...   boostedAt   
...                                                               ...   
xdai    0xff376c450192ebd1c911e6b8137919c80e3b291911bb...   boostedAt   
        0xff37f65a5cf37b109fc538ea2d18d2726f4c3c1cfdc3...  executedAt   
        0xff6c045d9ab8f514b7dce6317ac1ff16786cad69bd4a...   boostedAt   
        0xffc0f4a64c8ec043238ca787e58177c2e798360c2989...   boostedAt   
        0xffec994405570975de6a1e4fe71c4a3c5f909152fbdc...   boostedAt   

                                                                                                dao_q  \
network id                                                                                              
mainnet 0x0025c38d987acba1f1d446d3690384327ebe06d15f1f...  0x294f999356ed03347c7a23bcbcf8d33fa41dc830   
        0x0033a337b620add50be9ba0fff1128ab866ece51cfdd...  0x39757e7cdb7022c3829195d57b29428d79593c9d   
        0x003c109411e96e5607ce609278536262e12afdaa65ab...  0x519b70055af55a007110b4ff99b0ea33071c720a   
        0x0071c649968f7939fc9729c5dc25a8ae3f50dc86220a...  0x8990e11b69403ea53ef2b32434bbd7dbf84b5234   
        0x007eaf0fc8d6ae35ba3f28bc7cf4b34af659db8161d4...  0x440583455bcd85ab2bd429c015d3aabcae135f0a   
...                                                                                               ...   
xdai    0xff376c450192ebd1c911e6b8137919c80e3b291911bb...  0x94a587478c83491b13291265581cb983e7feb540   
        0xff37f65a5cf37b109fc538ea2d18d2726f4c3c1cfdc3...  0x4875e10

In [22]:
dfp['stakes'] = dfp['stakesFor'] / dfp['stakesAgainst']
dfp[dfp.closingAt < dfp.createdAt]

proposer  \
network id                                                                                              
mainnet 0x010c938f9cd050e3e3112fc1241a62583dec1ad09b9a...  0x91628ddc3a6ff9b48a2f34fc315d243eb07a9501   
        0x4e9d3e1ddd1e626d63a6f96018f955a2defa57200b91...  0x91628ddc3a6ff9b48a2f34fc315d243eb07a9501   
        0xd62fb5f9dfd7c98f9bc5e45700c374b40af1c59f8a34...  0x341a38b69a6b8f00fad4eed3a2b8528e3fed07ff   
        0xe8ef9ee91996f09af5a54deb675357cf66f217f3a190...  0xe3f7d7c3156a25435e74e8cb6587934e420d85ec   
xdai    0x05ac6f90c96c77898758d64c6ff2900bb5861084bdd6...  0x91628ddc3a6ff9b48a2f34fc315d243eb07a9501   
        0x1a813e3032d7ed14eecdd86d6c91fa43cf88d7b74744...  0xa5a29f81eee450ec189b2f8b4562af1785595d69   
        0x2497b482da5188cbd71cbc90dcceb1f3077142c55dab...  0x91aef3c3b9bab2c306548269ff9b6771f2b107d8   
        0x422fcfc674650f3807457b2ce5765342cc5eecdc9145...  0xa5a29f81eee450ec189b2f8b4562af1785595d69   
        0x4736aa5e9e97c24b0b8136e94b50e79a469355fadf41...  0x91628ddc3a6ff9b48a2f34fc315d243eb07a9501   
        0x487aec00c3c73e44b2b8ba8d4aaf317074f6022f460f...  0x91628ddc3a6ff9b48a2f34fc315d243eb07a9501   
        0x5061ddc2fd7542b5119b3e6f6c651c17ef561f1b1e92...  0x91aef3c3b9bab2c306548269ff9b6771f2b107d8   
        0x6b2b9da5d304c29d69b2e0b4db9f600c5cf41cd5a1e8...  0xa5a29f81eee450ec189b2f8b4562af1785595d69   
        0x718fe9257b9758c18ca9eef92466119a5c6d6a1e75e2...  0xe1d2210a967ee144aad31ecd08565e894b88ffaf   
        0x7427b0bfaaebc04f917253fa641c021ded395114efab...  0x8e900cf9bd655e34bb610f0ef365d8d476fd7337   
        0x8232b778e235f82167904e44934ed026b989186defcd...  0xe16d3664b313bd5fb8d911b467047e3cb4ed853d   
        0x88b8b7e41ea978cd9dad9fc9939ea56177813663d2d6...  0x91aef3c3b9bab2c306548269ff9b6771f2b107d8   
        0x8c87632f5dff6be7b2be3d5ef3a33554065201f7bdb4...  0x91628ddc3a6ff9b48a2f34fc315d243eb07a9501   
        0x991b4570a439520098f00d23755bce9e58a28d58ece5...  0x91aef3c3b9bab2c306548269ff9b6771f2b107d8   
        0x99ad0a5a48421ae7399c961f082d55c5bd2650c25f79...  0xa5a29f81eee450ec189b2f8b4562af1785595d69   
        0xa29e23024573899c0d38532b3ec48d4cfe9f5479d881...  0xa5a29f81eee450ec189b2f8b4562af1785595d69   
        0xaeba13d0ca82d212b6cea1464ba46aeab949068968a9...  0xa5a29f81eee450ec189b2f8b4562af1785595d69   
        0xbcae64cb284be394cafa2c3260a5edee3ec8baa118ca...  0xa5a29f81eee450ec189b2f8b4562af1785595d69   
        0xc7cf52f547b343d75c5c6c55238abf6e28702858e7dc...  0x8e900cf9bd655e34bb610f0ef365d8d476fd7337   
        0xc885ee1c06827b4922b8f3d1ddb4bb202a86b8874654...  0xa5a29f81eee450ec189b2f8b4562af1785595d69   
        0xd919ec25fe198f49cc8e0b32079a83fe165f35bcfd90...  0x35e2acd3f46b13151bc941daa44785a38f3bd97a   
        0xe8f4822489bedf5dec35f381f050da5b439156b4f403...  0x8e900cf9bd655e34bb610f0ef365d8d476fd7337   
        0xebdbce1b34db36d21ef282f356ae4778fbfcb5501b49...  0x91aef3c3b9bab2c306548269ff9b6771f2b107d8   
        0xffc0f4a64c8ec043238ca787e58177c2e798360c2989...  0xa601aed34dda12ff760d8abb64fd4eb3664e35af   

                                                              stage  \
network id                                                            
mainnet 0x010c938f9cd050e3e3112fc1241a62583dec1ad09b9a...  Executed   
        0x4e9d3e1ddd1e626d63a6f96018f955a2defa57200b91...  Executed   
        0xd62fb5f9dfd7c98f9bc5e45700c374b40af1c59f8a34...  Executed   
        0xe8ef9ee91996f09af5a54deb675357cf66f217f3a190...  Executed   
xdai    0x05ac6f90c96c77898758d64c6ff2900bb5861084bdd6...  Executed   
        0x1a813e3032d7ed14eecdd86d6c91fa43cf88d7b74744...  Executed   
        0x2497b482da5188cbd71cbc90dcceb1f3077142c55dab...  Executed   
        0x422fcfc674650f3807457b2ce5765342cc5eecdc9145...  Executed   
        0x4736aa5e9e97c24b0b8136e94b50e79a469355fadf41...  Executed   
        0x487aec00c3c73e44b2b8ba8d4aaf317074f6022f460f...  Executed   
        0x5061ddc2fd7542b5119b3e6f6c651c17ef561f1b1e92...  Executed   
        0x6b2b9da5d

# Preparing boosted proposals df

Una propuesta es boosteada en `boostedAt`. Una vez esta boosteada, 
- Cuando es ejecutada `executedAt`
- 

In [23]:
dfp.columns

Index(['proposer', 'stage', 'createdAt', 'preBoostedAt', 'boostedAt',
       'quietEndingPeriodBeganAt', 'closingAt', 'preBoostedClosingAt',
       'executedAt', 'totalRepWhenExecuted', 'totalRepWhenCreated',
       'executionState', 'expiresInQueueAt', 'votesFor', 'votesAgainst',
       'winningOutcome', 'stakesFor', 'stakesAgainst', 'title', 'description',
       'url', 'confidence', 'confidenceThreshold',
       'queuedVoteRequiredPercentage', 'queuedVotePeriodLimit',
       'boostedVotePeriodLimit', 'thresholdConst', 'minimumDaoBounty',
       'daoBountyConst', 'dao', 'competitionId', 'name', 'group',
       'originalName', 'votesForPct', 'votesAgainstPct', 'executedAtMonth',
       'createdAtMonth', 'daoRegister', 'nvoters', 'daostake', 'nstakers',
       'internalDAO', 'isProposer', 'stakes'],
      dtype='object')

In [24]:
print(dfp.columns)

def getPropB(row: pd.Series):
    boosted = row['boostedAt']
    unboosted = pd.NaT
    reason = ''
    
    if not pd.isna(boosted):
        unboosted = row['executedAt']
    
    return pd.Series({
        'boosted': boosted,
        'unboosted': unboosted,
        'delta': unboosted-boosted,
        'dao': row['dao'],
        'daoRegister': row['daoRegister'],
        'daoName': row['name'],
        'stage': row['stage'],
        'daostake': row['daostake'],
        'stakesFor': row['stakesFor'],
        'stakesAgainst': row['stakesAgainst'],
        'downstake': row['stakesAgainst'] - row['daostake'],
        'boostedVotePeriodLimit': row['boostedVotePeriodLimit'],
    }, name=row.name)

dfb = dfp.progress_apply(getPropB, axis='columns')
assert (dfb.boosted.isna() | dfb.unboosted.isna() | (dfb.boosted <= dfb.unboosted)).all()
%pickledf dfb

Index(['proposer', 'stage', 'createdAt', 'preBoostedAt', 'boostedAt',
       'quietEndingPeriodBeganAt', 'closingAt', 'preBoostedClosingAt',
       'executedAt', 'totalRepWhenExecuted', 'totalRepWhenCreated',
       'executionState', 'expiresInQueueAt', 'votesFor', 'votesAgainst',
       'winningOutcome', 'stakesFor', 'stakesAgainst', 'title', 'description',
       'url', 'confidence', 'confidenceThreshold',
       'queuedVoteRequiredPercentage', 'queuedVotePeriodLimit',
       'boostedVotePeriodLimit', 'thresholdConst', 'minimumDaoBounty',
       'daoBountyConst', 'dao', 'competitionId', 'name', 'group',
       'originalName', 'votesForPct', 'votesAgainstPct', 'executedAtMonth',
       'createdAtMonth', 'daoRegister', 'nvoters', 'daostake', 'nstakers',
       'internalDAO', 'isProposer', 'stakes'],
      dtype='object')


  0%|          | 0/5641 [00:00<?, ?it/s]

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 5641 entries, ('mainnet', '0x0025c38d987acba1f1d446d3690384327ebe06d15f1fa4171a4dc3467f8bd416') to ('xdai', '0xffec994405570975de6a1e4fe71c4a3c5f909152fbdcd7bc734b8dd2a094aa7f')
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype          
---  ------                  --------------  -----          
 0   boosted                 3710 non-null   datetime64[ns] 
 1   unboosted               3704 non-null   datetime64[ns] 
 2   delta                   3704 non-null   timedelta64[ns]
 3   dao                     5641 non-null   object         
 4   daoRegister             5641 non-null   object         
 5   daoName                 3967 non-null   object         
 6   stage                   5641 non-null   object         
 7   daostake                5641 non-null   float64        
 8   stakesFor               5641 non-null   float64        
 9   stakesAgainst           5641 non-null   float64        
 10  do

boosted  \
network id                                                                      
mainnet 0x0025c38d987acba1f1d446d3690384327ebe06d15f1f... 2019-10-23 12:44:17   
        0x0033a337b620add50be9ba0fff1128ab866ece51cfdd...                 NaT   
        0x003c109411e96e5607ce609278536262e12afdaa65ab... 2020-03-15 19:54:03   
        0x0071c649968f7939fc9729c5dc25a8ae3f50dc86220a... 2019-12-15 21:57:46   
        0x007eaf0fc8d6ae35ba3f28bc7cf4b34af659db8161d4... 2019-11-03 07:48:08   
...                                                                       ...   
xdai    0xff376c450192ebd1c911e6b8137919c80e3b291911bb... 2021-04-22 16:26:50   
        0xff37f65a5cf37b109fc538ea2d18d2726f4c3c1cfdc3...                 NaT   
        0xff6c045d9ab8f514b7dce6317ac1ff16786cad69bd4a... 2022-02-09 10:51:15   
        0xffc0f4a64c8ec043238ca787e58177c2e798360c2989... 2021-01-23 21:10:55   
        0xffec994405570975de6a1e4fe71c4a3c5f909152fbdc... 2021-05-07 03:20:10   

                                                                    unboosted  \
network id                                                                      
mainnet 0x0025c38d987acba1f1d446d3690384327ebe06d15f1f... 2019-10-27 12:44:18   
        0x0033a337b620add50be9ba0fff1128ab866ece51cfdd...                 NaT   
        0x003c109411e96e5607ce609278536262e12afdaa65ab... 2020-03-22 19:54:21   
        0x0071c649968f7939fc9729c5dc25a8ae3f50dc86220a... 2019-12-18 21:57:59   
        0x007eaf0fc8d6ae35ba3f28bc7cf4b34af659db8161d4... 2019-11-06 09:25:53   
...                                                                       ...   
xdai    0xff376c450192ebd1c911e6b8137919c80e3b291911bb... 2021-04-30 19:17:55   
        0xff37f65a5cf37b109fc538ea2d18d2726f4c3c1cfdc3...                 NaT   
        0xff6c045d9ab8f514b7dce6317ac1ff16786cad69bd4a... 2022-02-13 10:51:30   
        0xffc0f4a64c8ec043238ca787e58177c2e798360c2989... 2021-01-27 21:11:00   
        0xffec994405570975de6a1e4fe71c4a3c5f909152fbdc... 2021-05-11 03:20:15   

                                                                    delta  \
network id                                                                  
mainnet 0x0025c38d987acba1f1d446d3690384327ebe06d15f1f... 4 days 00:00:01   
        0x0033a337b620add50be9ba0fff1128ab866ece51cfdd...             NaT   
        0x003c109411e96e5607ce609278536262e12afdaa65ab... 7 days 00:00:18   
        0x0071c649968f7939fc9729c5dc25a8ae3f50dc86220a... 3 days 00:00:13   
        0x007eaf0fc8d6ae35ba3f28bc7cf4b34af659db8161d4... 3 days 01:37:45   
...                                                                   ...   
xdai    0xff376c450192ebd1c911e6b8137919c80e3b291911bb... 8 days 02:51:05   
        0xff37f65a5cf37b109fc538ea2d18d2726f4c3c1cfdc3...             NaT   
        0xff6c045d9ab8f514b7dce6317ac1ff16786cad69bd4a... 4 days 00:00:15   
        0xffc0f4a64c8ec043238ca787e58177c2e798360c2989... 4 days 00:00:05   
        0xffec994405570975de6a1e4fe71c4a3c5f909152fbdc... 4 days 00:00:05   

                                                                                                  dao  \
network id                                                                                              
mainnet 0x0025c38d987acba1f1d446d3690384327ebe06d15f1f...  0x294f999356ed03347c7a23bcbcf8d33fa41dc830   
        0x0033a337b620add50be9ba0fff1128ab866ece51cfdd...  0x39757e7cdb7022c3829195d57b29428d79593c9d   
        0x003c109411e96e5607ce609278536262e12afdaa65ab...  0x519b70055af55a007110b4ff99b0ea33071c720a   
        0x0071c649968f7939fc9729c5dc25a8ae3f50dc86220a...  0x8990e11b69403ea53ef2b32434bbd7dbf84b5234   
        0x007eaf0fc8d6ae35ba3f28bc7cf4b34af659db8161d4...  0x440583455bcd85ab2bd429c015d3aabcae135f0a   
...                                                                                               ...   
xdai    0xff376c450192ebd1c911e6b8137919c80e3b291911bb...  0x94a587478c83491b13291265581cb983e7feb540   
        0xff37f

## Preparing boosted count df (dfbcnt)

In [25]:
cols = ['at', 'network', 'id', 'dao', 'daoRegister', 'daoName', 'stakesFor', 'stakesAgainst', 'downstake']

# TODO: Shouldn't the index be date, network, id
# And if only date, network is needed, the cnt should be the sum of internal cnt
dfboosted = dfb.dropna(subset='boosted').reset_index().rename(columns={'boosted': 'at'})[cols]
dfunboosted = dfb.dropna(subset='unboosted').reset_index().rename(columns={'unboosted': 'at'})[cols]
dfboosted['cnt'] = 1
dfboosted['stakesAgainst'] *= -1
dfunboosted['cnt'] = -1
dfunboosted['stakesFor'] *= -1

dfbcnt = pd.concat([dfboosted, dfunboosted]).sort_values('at')
%pickledf dfbcnt

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7414 entries, 1229 to 1903
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   at             7414 non-null   datetime64[ns]
 1   network        7414 non-null   category      
 2   id             7414 non-null   string        
 3   dao            7414 non-null   object        
 4   daoRegister    7414 non-null   object        
 5   daoName        5720 non-null   object        
 6   stakesFor      7414 non-null   float64       
 7   stakesAgainst  7414 non-null   float64       
 8   downstake      7414 non-null   float64       
 9   cnt            7414 non-null   int64         
dtypes: category(1), datetime64[ns](1), float64(3), int64(1), object(3), string(1)
memory usage: 586.6+ KB
None
dfbcnt pickled to .pickles/dfbcnt.pickle


,at,network,id,dao,daoRegister,daoName,stakesFor,stakesAgainst,downstake,cnt
1229,2019-04-12 08:47:05,mainnet,0xc4a445241e97d04128620bfc2bfecdfecdfbdf0f79d6...,0x294f999356ed03347c7a23bcbcf8d33fa41dc830,registered,Genesis Alpha,201.0,-150.0,0.0,1
1369,2019-04-12 15:23:42,mainnet,0xdc67e00f7bf1cb385bdb98cbf7a5c4e021ea36ea495d...,0x294f999356ed03347c7a23bcbcf8d33fa41dc830,registered,Genesis Alpha,301.0,-150.0,0.0,1
633,2019-04-14 12:13:22,mainnet,0x68cfc07c5745b88a1d89b6446c3d9164bdc1cb1d3dbb...,0x294f999356ed03347c7a23bcbcf8d33fa41dc830,registered,Genesis Alpha,220.0,-150.0,0.0,1
1225,2019-04-16 10:03:01,mainnet,0xc4a445241e97d04128620bfc2bfecdfecdfbdf0f79d6...,0x294f999356ed03347c7a23bcbcf8d33fa41dc830,registered,Genesis Alpha,-201.0,150.0,0.0,-1
1365,2019-04-16 19:58:09,mainnet,0xdc67e00f7bf1cb385bdb98cbf7a5c4e021ea36ea495d...,0x294f999356ed03347c7a23bcbcf8d33fa41dc830,registered,Genesis Alpha,-301.0,150.0,0.0,-1
...,...,...,...,...,...,...,...,...,...,...
1908,2023-04-07 09:34:55,xdai,0x2595e6bff6eb185088b709bd1c9a7058919ca22caed6...,0xe716ec63c5673b3a4732d22909b38d779fa47c3f,registered,xDXdao,500.0,-200.0,0.0,1
83,2023-04-07 15:36:11,mainnet,0x0dc340bf6583557c718716d50d6fc8adbc6f73ab7414...,0x519b70055af55a007110b4ff99b0ea33071c720a,registered,dxDAO,3500.0,-1000.0,0.0,1
270,2023-04-08 13:37:47,mainnet,0x2f701be3fbd3e7e706f1aac4d36839c18a56a52e45e5...,0x519b70055af55a007110b4ff99b0ea33071c720a,registered,dxDAO,-750.0,250.0,0.0,-1
82,2023-04-09 11:34:47,mainnet,0x0dc340bf6583557c718716d50d6fc8adbc6f73ab7414...,0x519b70055af55a007110b4ff99b0ea33071c720a,registered,dxDAO,-3500.0,1000.0,0.0,-1


In [26]:
print("Currently boosted proposals:", dfbcnt['cnt'].sum())

Currently boosted proposals: 6


## Check integrity

In [27]:
dfbo = dfb.join(dfo, lsuffix='_b', rsuffix='_o')
dfbq = dfb.join(dfq, lsuffix='_b', rsuffix='_q')

assert (dfbo.boosted.isna() | ((dfbo.opened <= dfbo.boosted) & (dfbo.boosted <= dfbo.closed))).all(), "A proposal should be open before boosted"
assert (dfbo.unboosted.isna() | (dfbo.unboosted == dfbo.closed)).all(), "A proposal is closed when unboosted"
assert (dfbq.boosted.isna() | (dfbq.boosted == dfbq.unqueued)).all(), "A proposal is unqueued when boosted"
dfbo[(~dfbo.unboosted.isna()) & (dfbo.reason != 'executedAt')]

,,boosted,unboosted,delta_b,dao_b,daoRegister_b,daoName_b,stage,daostake,stakesFor,stakesAgainst,downstake,boostedVotePeriodLimit,opened,closed,reason,dao_o,daoRegister_o,daoName_o,delta_o
network,id,,,,,,,,,,,,,,,,,,,


# Preparing holders df

In [28]:
dfh = pd.read_feather(DAOSTACK.REP_HOLDERS)
dfh = utils.append_dao_names(dfh)
dfh = utils.dfAtToDatetime(dfh)
dfh = dfh.astype({
    'network': 'category',
    'address': 'string',
    'dao': 'category',
}).convert_dtypes()
dfh['balance'] = dfh['balance'].astype(float) / 10**18

# Adding some aggregated data
dfh = dfh.set_index(['network', 'dao', 'address'])

dfh['nvotes'] = pd.read_feather(DAOSTACK.VOTES).groupby(['network', 'dao', 'voter'])['id'].count()
dfh['nvotes'] = dfh['nvotes'].fillna(0).astype(int)

dfh['nproposals'] = pd.read_feather(DAOSTACK.PROPOSALS).groupby(['network', 'dao', 'proposer'])['id'].count()
dfh['nproposals'] = dfh['nproposals'].fillna(0).astype(int)

stkgby = dfs.groupby(['network', 'dao', 'staker'])

dfh['nstakes'] = stkgby['id'].count()
dfh['nstakes'] = dfh['nstakes'].fillna(0).astype(int)
dfh['stakeAmount'] = stkgby['amount'].apply(lambda s: s.abs().sum())
dfh['stakeAmount'] = dfh['stakeAmount'].fillna(0)
dfh['nact'] = dfh['nvotes'] + dfh['nproposals'] + dfh['nstakes']

dfh = dfh.reset_index()

%pickledf dfh

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14344 entries, 0 to 14343
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   network       14344 non-null  category      
 1   dao           14344 non-null  category      
 2   address       14344 non-null  string        
 3   id            14344 non-null  string        
 4   contract      14344 non-null  string        
 5   balance       14344 non-null  float64       
 6   createdAt     14344 non-null  datetime64[ns]
 7   name          9626 non-null   string        
 8   group         9626 non-null   string        
 9   originalName  9626 non-null   string        
 10  nvotes        14344 non-null  int64         
 11  nproposals    14344 non-null  int64         
 12  nstakes       14344 non-null  int64         
 13  stakeAmount   14344 non-null  float64       
 14  nact          14344 non-null  int64         
dtypes: category(2), datetime64[ns](1), f

,network,dao,address,id,contract,balance,createdAt,name,group,originalName,nvotes,nproposals,nstakes,stakeAmount,nact
0,mainnet,0x6bee9b81e434f7afce72a43a4016719315069539,0xbe5cf9a0408d22cdd61f8990b33dd00a5272f65b,0x0003a06b89cb93c8f3c630c0c26f64b13e1c844ee224...,0xed1c0bb038b370a16242f077899ee1e5e7d6add1,30.00,2019-06-20 14:11:02,Kyber DAO Exp#2,Kyber DAO Exp#2,Kyber DAO Exp#2,0,0,0,0.0,0
1,mainnet,0x273d0f686a53a49a0fba2a801566b19f9aaf8fcd,0xe0d6480ae69e7c4928a3f45e6cf5651e52741ec0,0x001688e16afd026c2f4f943a0c0e20a07591e6b0be45...,0x4352a94ccb3ea07e16734f66a94e94e474a50b5f,50.00,2020-12-01 19:01:19,PrimeDAO,PrimeDAO,PrimeDAO,0,0,0,0.0,0
2,mainnet,0x6bee9b81e434f7afce72a43a4016719315069539,0xbf4e9e59b1ac3a1d0df596ce68841ee7c5008388,0x0018946eee358a93d9b8a681344621950128ae20357c...,0xed1c0bb038b370a16242f077899ee1e5e7d6add1,10.00,2019-06-20 13:02:35,Kyber DAO Exp#2,Kyber DAO Exp#2,Kyber DAO Exp#2,0,0,0,0.0,0
3,mainnet,0x6bee9b81e434f7afce72a43a4016719315069539,0xddf2bb10b39bbc89746c8b81e7d08922fbbcc9aa,0x001d02d26aa50572b3c67e6c60a1ca7b60e1168aa6be...,0xed1c0bb038b370a16242f077899ee1e5e7d6add1,50.00,2019-06-20 12:42:17,Kyber DAO Exp#2,Kyber DAO Exp#2,Kyber DAO Exp#2,0,0,0,0.0,0
4,mainnet,0x6bee9b81e434f7afce72a43a4016719315069539,0x705dc139f523c0056d0a7e3b80799ddaf0b8b640,0x0030fa6c205655268a0ccd7587c22c2b0df971bea864...,0xed1c0bb038b370a16242f077899ee1e5e7d6add1,10.00,2019-06-20 12:58:55,Kyber DAO Exp#2,Kyber DAO Exp#2,Kyber DAO Exp#2,0,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14339,xdai,0xd548f7c3814603a6b588326c37f925dfdfb396bf,0xc5863148cfa940422f50e1576e3825bf5bd1884d,0xffbfe05e0502ad0e20fcf03365ea8cfde7b0a17d8f8a...,0xcbedf512b279deaaf62967c832aff75a1e39d07b,0.11,2020-11-17 08:43:45,<NA>,<NA>,<NA>,0,0,0,0.0,0
14340,xdai,0x104e3ff64ac66ea70b90c53c8d63eb63a73b1103,0xc820b0905a8ae142fcc012a6000abb5c0a84f522,0xffc35a10bea46344bef37bce7df4421bfe2c27c19627...,0x61a82e67f1fd9a86febb8326dc74577a2cc766cc,100.00,2022-01-09 13:32:15,<NA>,<NA>,<NA>,0,0,0,0.0,0
14341,xdai,0xe248a76a4a84667c859eb51b9af6dea29e52f139,0xba32e9ec605741ccbb1053936f35732814b38415,0xffeb7cf2efc4bb21e494b787bc4b391293d03c817274...,0x7905f89e877e16822c46fe7509bb61a6e0ae3af1,1.00,2020-02-08 22:09:55,BuffiDAO,BuffiDAO,BuffiDAO,0,0,0,0.0,0
14342,xdai,0xe248a76a4a84667c859eb51b9af6dea29e52f139,0xe83d8c8410efeea34181c65e978b9cc25dbe02e6,0xfff39d1d9f1651fdccabd49b37ad189ef67ac0730a8c...,0x7905f89e877e16822c46fe7509bb61a6e0ae3af1,1.00,2020-02-08 21:58:05,BuffiDAO,BuffiDAO,BuffiDAO,0,0,0,0.0,0


# Preparing Activity df (dfa)

Columns:
- network
- DAO
- kind (voting / proposing /staking )
- at (date of activity)
- id (id from original df)
- user (the user who voted / made the proposal / staked)

In [29]:
DAO_ACTIVITY_COLS = ['network', 'dao', 'at', 'kind', 'id', 'user']

dfpa = pd.read_feather(DAOSTACK.PROPOSALS)
dfpa = dfpa.rename(columns={'createdAt': 'at', 'proposer': 'user'})
dfpa['kind'] = 'proposing'

dfva = pd.read_feather(DAOSTACK.VOTES)
dfva = dfva.rename(columns={'createdAt': 'at', 'voter': 'user'})
dfva['kind'] = 'voting'

dfsa = pd.read_feather(DAOSTACK.STAKES)
dfsa = dfsa.rename(columns={'createdAt': 'at', 'staker': 'user'})
dfsa['kind'] = 'staking'

dfa = pd.concat([x[DAO_ACTIVITY_COLS] for x in [dfpa, dfva, dfsa]]).sort_values('at')
dfa['at'] = utils.colAutoDatetime(dfa['at'])
dfa = dfa.astype({
    'network': 'category',
    'kind': 'category',
    'dao': 'category',
    'user': 'category',
})
%pickledf dfa

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31731 entries, 1158 to 3894
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   network  31731 non-null  category      
 1   dao      31731 non-null  category      
 2   at       31731 non-null  datetime64[ns]
 3   kind     31731 non-null  category      
 4   id       31731 non-null  object        
 5   user     31731 non-null  category      
dtypes: category(4), datetime64[ns](1), object(1)
memory usage: 981.7+ KB
None
dfa pickled to .pickles/dfa.pickle


,network,dao,at,kind,id,user
1158,mainnet,0x294f999356ed03347c7a23bcbcf8d33fa41dc830,2019-04-10 18:59:04,proposing,0x748b23205728476ea4d3e9d3a752bc1f7fdaadf660e9...,0xf3215bf9ad166213e2588baf81d7e932fda99d9e
1841,mainnet,0x294f999356ed03347c7a23bcbcf8d33fa41dc830,2019-04-10 18:59:11,proposing,0xb775ceea172980b5130db7eb24d07fe9713338845c1b...,0xf3215bf9ad166213e2588baf81d7e932fda99d9e
2407,mainnet,0x294f999356ed03347c7a23bcbcf8d33fa41dc830,2019-04-10 19:01:47,proposing,0xf038967ac6e37121f54d5d4168df9d717282d63fac3f...,0x144c4e5027b69f7798b2b162d924bcae5c149f15
2902,mainnet,0x294f999356ed03347c7a23bcbcf8d33fa41dc830,2019-04-10 19:38:39,staking,0xe7b99994b2983ed1ec6f3ea44ccab4c2c429428d0118...,0x144c4e5027b69f7798b2b162d924bcae5c149f15
6774,mainnet,0x294f999356ed03347c7a23bcbcf8d33fa41dc830,2019-04-10 19:38:39,voting,0xe6500e8c1d372a001cf7860c7f8c1d0583bd554fd346...,0x144c4e5027b69f7798b2b162d924bcae5c149f15
...,...,...,...,...,...,...
6874,mainnet,0x519b70055af55a007110b4ff99b0ea33071c720a,2023-04-08 07:38:59,voting,0xe9e8dc42ff8ab7a1774c8c66e5289da4b24575dbab60...,0x583acc79585d3cb195ea8125f6f80ad459b46313
1559,mainnet,0x519b70055af55a007110b4ff99b0ea33071c720a,2023-04-08 13:45:11,voting,0x36a35b7483e3fca93ba570656a6540f756f6bb7780e1...,0x7e72cfd9a36517435dc1ca7f9451eccbc973111e
2306,mainnet,0x519b70055af55a007110b4ff99b0ea33071c720a,2023-04-08 18:13:23,voting,0x4fb9b42a55c7219c906a6cd674f53e692e81b1083304...,0x3111327edd38890c3fe564afd96b4c73e8101747
2285,mainnet,0x519b70055af55a007110b4ff99b0ea33071c720a,2023-04-09 08:40:35,voting,0x4ef0de5bb381268273ca1aa194cb7c1a0c0b42c4a14c...,0x759a2169da1b826f795a00a9ab5f29f9ca39e48a


In [30]:
dfa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31731 entries, 1158 to 3894
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   network  31731 non-null  category      
 1   dao      31731 non-null  category      
 2   at       31731 non-null  datetime64[ns]
 3   kind     31731 non-null  category      
 4   id       31731 non-null  object        
 5   user     31731 non-null  category      
dtypes: category(4), datetime64[ns](1), object(1)
memory usage: 981.7+ KB


# Preparing group df (dfg)

In [31]:
def _apply_network(x):
        n = len(set(x))
        
        if n == 0:
            return "?"
        elif n == 1:
            return list(x)[0]
        else:
            return "mixed"

def calculate_dfg(_dfd):
    gby = _dfd.groupby('group')
    dfg = gby['group'].count().rename('daocnt').to_frame().sort_index()
        
    _dfpg = utils.get_df('dfp').reset_index().groupby('group')
    _dfvg = pd.read_feather(DAOSTACK.VOTES).ds.append_dao_info().groupby('group')
    
    dfg['firstActivity'] = _dfd.groupby('group')['firstActivity'].min()
    dfg['lastProposal'] = _dfpg['createdAt'].max()
    dfg['diff'] = np.ceil((dfg['lastProposal'] - dfg['firstActivity'])/np.timedelta64(1, 'M'))
    dfg['network'] = gby['network'].apply(_apply_network)
    dfg['nusers'] = dfh.groupby('group')['address'].nunique()
    dfg['hnusers'] = utils.append_dao_info(pd.read_feather(DAOSTACK.REP_MINTS)).groupby('group')['address'].nunique()
    dfg['hnusers'] = dfg['hnusers'].fillna(0)
    dfg['nproposals'] = _dfpg['id'].count()
    dfg['nvotes'] = _dfvg['id'].count()
    dfg['nvoters'] = _dfvg['voter'].apply(utils.get_nunique_in_group)
    dfg['nvoters_pct'] = dfg['nvoters'] / dfg['hnusers']
    dfg['nproposers'] = _dfpg['proposer'].apply(utils.get_nunique_in_group)
    dfg['pctApproved'] = _dfpg['winningOutcome'].apply(lambda x: np.mean(x == 'Pass'))
    dfg['pctBoosted'] = _dfpg['boostedAt'].apply(lambda x: np.mean(~pd.isna(x)))
    # dfg['vpp'] = _dfpg['nvoters'].mean()
    
    dfg['vpp'] = _dfvg['id'].count() / _dfpg['id'].count()
    dfg['spp'] = pd.read_feather(DAOSTACK.STAKES).ds.append_dao_info().groupby('group')['id'].count() / _dfpg['id'].count()
    dfg['spp'] = dfg['spp'].fillna(0)
    
    dfp = utils.get_df('dfp')
    dfp['approved'] = dfp['winningOutcome'] == 'Pass'
    dfp['staked'] = dfp['stakesFor'].astype(float) > 0
    dfp['boosted'] = ~dfp['boostedAt'].isna()
    dfp['TP'] =  dfp['boosted'] &  dfp['approved']
    dfp['TN'] = ~dfp['boosted'] & ~dfp['approved']
    dfp['FP'] =  dfp['boosted'] & ~dfp['approved']
    dfp['FN'] = ~dfp['boosted'] &  dfp['approved']
    
    _dfpg = dfp.groupby('group')
    dfg[['TP', 'TN', 'FP', 'FN']] = _dfpg[['TP', 'TN', 'FP', 'FN']].sum()
    dfg['precision'] = (dfg['TP'] + dfg['TN']) / dfg[['TP', 'TN', 'FP', 'FN']].sum(axis=1)
    dfg['precision_boosted'] = dfg['TP'] / (dfg['TP'] + dfg['FP'])
    dfg['precision_nonboosted'] = dfg['TN'] / (dfg['TN'] + dfg['FN'])
    
    assert dfg['daocnt'].sum() == len(_dfd), f"{dfg['daocnt'].sum()} != {len(_dfd)}"
    assert dfg.loc['dxDAO']['network'] == 'mixed'
    assert dfg.loc['dxDAO']['daocnt'] == 2
    assert dfg.loc['dOrg']['network'] == 'mainnet'
    assert dfg.loc['dOrg']['daocnt'] == 3

    return dfg

dfg = calculate_dfg(utils.dropDAOs(utils.get_df('dfd')).reset_index())
%pickledf dfg

<class 'pandas.core.frame.DataFrame'>
Index: 23 entries, 1UP to necDAO
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   daocnt                23 non-null     int64         
 1   firstActivity         23 non-null     datetime64[ns]
 2   lastProposal          23 non-null     datetime64[ns]
 3   diff                  23 non-null     float64       
 4   network               23 non-null     object        
 5   nusers                23 non-null     int64         
 6   hnusers               23 non-null     int64         
 7   nproposals            23 non-null     int64         
 8   nvotes                23 non-null     int64         
 9   nvoters               23 non-null     int64         
 10  nvoters_pct           23 non-null     float64       
 11  nproposers            23 non-null     int64         
 12  pctApproved           23 non-null     float64       
 13  pctBoosted           

,daocnt,firstActivity,lastProposal,diff,network,nusers,hnusers,nproposals,nvotes,nvoters,nvoters_pct,nproposers,pctApproved,pctBoosted,vpp,spp,TP,TN,FP,FN,precision,precision_boosted,precision_nonboosted
group,,,,,,,,,,,,,,,,,,,,,,,
1UP,1,2020-03-23 21:50:30,2020-10-02 19:22:26,7.0,mainnet,14,14,40,33,4,0.285714,14,0.675000,0.700000,0.825000,1.175000,27,12,1,0,0.975000,0.964286,1.000000
3Box,1,2020-02-10 20:35:50,2020-06-06 19:15:45,4.0,xdai,5,5,8,15,3,0.600000,0,0.625000,0.000000,1.875000,0.000000,0,3,0,5,0.375000,NaN,0.375000
BuffiDAO,1,2020-02-06 18:29:00,2020-12-23 10:03:06,11.0,xdai,2730,2830,25,22,4,0.001413,6,0.640000,0.000000,0.880000,0.080000,0,9,0,16,0.360000,NaN,0.360000
CENNZnet Grants DAO,1,2020-04-20 11:00:14,2020-08-19 14:58:02,4.0,mainnet,12,12,21,30,8,0.666667,9,0.904762,0.904762,1.428571,1.714286,19,2,0,0,1.000000,1.000000,1.000000
CuraDAO,1,2019-08-06 22:47:40,2021-08-18 06:44:29,25.0,mainnet,55,55,49,103,11,0.200000,12,0.795918,0.836735,2.102041,1.163265,39,8,2,0,0.959184,0.951220,1.000000
DAOfund,1,2019-07-31 19:57:13,2021-05-29 10:02:37,22.0,mainnet,6,8,49,60,7,0.875000,7,0.877551,0.469388,1.224490,0.591837,20,3,3,23,0.469388,0.869565,0.115385
DAOstack DAO,1,2020-02-12 21:49:20,2020-11-27 15:31:45,10.0,xdai,6,6,13,15,4,0.666667,1,0.153846,0.000000,1.153846,0.153846,0,11,0,2,0.846154,NaN,0.846154
DetroitDAO,1,2019-10-15 10:31:52,2020-01-01 09:35:51,3.0,mainnet,6,6,11,23,3,0.500000,2,0.909091,0.000000,2.090909,0.000000,0,1,0,10,0.090909,NaN,0.090909
ETHGlobal,1,2019-11-07 19:30:21,2019-12-07 00:40:40,1.0,mainnet,48,48,19,74,24,0.500000,13,0.368421,0.315789,3.894737,2.578947,5,11,1,2,0.842105,0.833333,0.846154


In [32]:
dfgu = calculate_dfg(utils.get_df('dfd').reset_index())
%pickledf dfgu

<class 'pandas.core.frame.DataFrame'>
Index: 68 entries, 1UP to xDAItestDAO
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   daocnt                68 non-null     int64         
 1   firstActivity         61 non-null     datetime64[ns]
 2   lastProposal          61 non-null     datetime64[ns]
 3   diff                  61 non-null     float64       
 4   network               68 non-null     object        
 5   nusers                68 non-null     int64         
 6   hnusers               68 non-null     int64         
 7   nproposals            61 non-null     float64       
 8   nvotes                60 non-null     float64       
 9   nvoters               60 non-null     float64       
 10  nvoters_pct           60 non-null     float64       
 11  nproposers            61 non-null     float64       
 12  pctApproved           61 non-null     float64       
 13  pctBoosted      

,daocnt,firstActivity,lastProposal,diff,network,nusers,hnusers,nproposals,nvotes,nvoters,nvoters_pct,nproposers,pctApproved,pctBoosted,vpp,spp,TP,TN,FP,FN,precision,precision_boosted,precision_nonboosted
group,,,,,,,,,,,,,,,,,,,,,,,
1UP,1,2020-03-23 21:50:30,2020-10-02 19:22:26,7.0,mainnet,14,14,40.0,33.0,4.0,0.285714,14.0,0.675000,0.700000,0.825000,1.175000,27.0,12.0,1.0,0.0,0.975000,0.964286,1.000000
3Box,2,2020-02-10 20:35:50,2020-06-06 19:15:45,4.0,xdai,5,5,8.0,15.0,3.0,0.600000,0.0,0.625000,0.000000,1.875000,0.000000,0.0,3.0,0.0,5.0,0.375000,NaN,0.375000
Advance Colorado DAO,1,2020-02-13 19:14:15,2020-12-17 10:55:00,11.0,xdai,2,2,3.0,2.0,1.0,0.500000,1.0,0.666667,0.000000,0.666667,0.000000,0.0,1.0,0.0,2.0,0.333333,NaN,0.333333
Arbitrum DAO,1,2020-02-12 02:37:25,2020-02-12 02:37:25,0.0,xdai,1,1,1.0,1.0,0.0,0.000000,0.0,1.000000,0.000000,1.000000,0.000000,0.0,0.0,0.0,1.0,0.000000,NaN,0.000000
Bancor DAO,1,2020-02-13 15:13:00,2020-02-13 15:13:00,0.0,xdai,1,1,1.0,1.0,0.0,0.000000,0.0,1.000000,0.000000,1.000000,0.000000,0.0,0.0,0.0,1.0,0.000000,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eth2,1,2020-02-13 15:47:10,2020-02-13 15:53:25,1.0,xdai,1,1,2.0,2.0,0.0,0.000000,0.0,0.500000,0.000000,1.000000,0.000000,0.0,1.0,0.0,1.0,0.500000,NaN,0.500000
iExec DAO,3,2020-02-12 02:56:35,2020-02-12 02:58:40,1.0,xdai,3,3,2.0,4.0,2.0,0.666667,0.0,1.000000,0.000000,2.000000,0.000000,0.0,0.0,0.0,2.0,0.000000,NaN,0.000000
necDAO,1,2020-01-26 12:26:36,2023-02-20 14:05:47,37.0,mainnet,140,140,43.0,137.0,39.0,0.278571,11.0,0.651163,0.767442,3.186047,1.209302,28.0,10.0,5.0,0.0,0.883721,0.848485,1.000000
